### Clusters no Jerárquicos

Una liga internacional de **e-sports** quiere segmentar a sus equipos profesionales para diseñar programas de entrenamiento personalizados, estrategias de balanceo competitivo (emparejamientos, divisiones, etc.) y estrategias de contenido y retransmisión en plataformas de streaming.

Para ello se ha recopilado información de **$18$ equipos profesionales** de un mismo videojuego competitivo. Las variables disponibles son:

+ `Equipo`: identificador del equipo.

+ `Win_rate`: porcentaje de partidas ganadas en la última temporada (%).

+ `KDA`: ratio medio de bajas/asistencias frente a muertes (Kill/Death/Assist ratio).

+ `Control_obj`: porcentaje de control de objetivos clave del mapa (torres, dragones, puntos de captura, etc.) (%).

+ `Entreno_horas`: horas medias de entrenamiento en equipo por semana.

El **objetivo** es aplicar técnicas de clustering no jerárquico para identificar perfiles de equipos (por ejemplo, equipos hiperentrenados, equipos muy eficientes en objetivos, equipos dependientes del “early game”, etc.). La base de datos se resume en la siguiente tabla:

| Equipo  | Win_rate | KDA  | Control_obj | Entreno_horas |
| ------- | -------- | ---- | ----------- | ------------- |
| Team_01 | 69.6     | 3.93 | 66.0        | 31            |
| Team_02 | 66.5     | 4.21 | 49.9        | 30            |
| Team_03 | 54.7     | 2.43 | 71.2        | 18            |
| Team_04 | 49.1     | 3.69 | 63.4        | 24            |
| Team_05 | 57.9     | 4.20 | 30.1        | 32            |
| Team_06 | 54.2     | 3.55 | 54.7        | 24            |
| Team_07 | 67.4     | 2.92 | 73.4        | 37            |
| Team_08 | 50.6     | 1.80 | 42.2        | 18            |
| Team_09 | 56.7     | 2.80 | 46.3        | 32            |
| Team_10 | 60.4     | 3.33 | 73.5        | 25            |
| Team_11 | 71.8     | 4.24 | 39.6        | 32            |
| Team_12 | 57.7     | 4.40 | 58.4        | 21            |
| Team_13 | 49.9     | 2.93 | 41.9        | 40            |
| Team_14 | 66.5     | 4.10 | 78.4        | 34            |
| Team_15 | 61.6     | 2.28 | 70.2        | 32            |
| Team_16 | 48.8     | 3.92 | 52.4        | 33            |
| Team_17 | 71.8     | 3.15 | 34.0        | 24            |
| Team_18 | 74.4     | 1.54 | 46.0        | 29            |

Realizar los siguientes apartados:

1. Crear la base de datos a partir de dicha información y realizar el análisis exploratorio.
2. Realizar el análisis de la existencia de relación y semejanza entre las variables.
3. Calcular la matriz de proximidad utilizando la distancia euclidiana.
4. Estimar e interpretar los modelos de clustering no jerárquicos considerando los cuatro métodos.
5. Calcular el número de clústers óptimos para cada uno de los cuatro métodos a partir de los tres criterios vistos.

<div style="background-color:#5DADE2"><b>Apartado 1</b></div>

In [ ]:
# Crear el data frame con los datos indicados
wdata <- data.frame(
  Equipo = paste0("Team_", sprintf("%02d", 1:18)),
  Win_rate = c(69.6, 66.5, 54.7, 49.1, 57.9, 54.2, 67.4, 50.6, 56.7,
               60.4, 71.8, 57.7, 49.9, 66.5, 61.6, 48.8, 71.8, 74.4),
  KDA = c(3.93, 4.21, 2.43, 3.69, 4.20, 3.55, 2.92, 1.80, 2.80,
          3.33, 4.24, 4.40, 2.93, 4.10, 2.28, 3.92, 3.15, 1.54),
  Control_obj = c(66.0, 49.9, 71.2, 63.4, 30.1, 54.7, 73.4, 42.2, 46.3,
                  73.5, 39.6, 58.4, 41.9, 78.4, 70.2, 52.4, 34.0, 46.0),
  Entreno_horas = c(31, 30, 18, 24, 32, 24, 37, 18, 32,
                    25, 32, 21, 40, 34, 32, 33, 24, 29)
)

wdata


In [ ]:
# Realizamos el análisis exploratorio de las variables utilizadas
summary(wdata)

In [ ]:
# Cargar librería necesaria para kurtosis
if(!require(moments, quietly = TRUE)){
  install.packages("moments", repos = "https://cloud.r-project.org")
  library(moments)
}

# Desviación estándar de todas las variables numéricas
sapply(wdata, sd, na.rm = TRUE)

In [ ]:
# Curtosis de todas las variables numéricas
sapply(wdata[sapply(wdata, is.numeric)], kurtosis, na.rm = TRUE)


**<u>Interpretación resultados</u>.-**

La base de datos contiene información de **18 equipos profesionales de e-sports** y cinco variables de las cuales cuatro cuantitativas que describen su rendimiento competitivo y su volumen de entrenamiento, sin presencia de valores atípicos ni registros anómalos. A continuación se interpretan los estadísticos descriptivos de cada variable en su contexto.

**Equipo** actúa como identificador único, con 18 equipos diferentes incluidos en el análisis.

**Win_rate** recoge el porcentaje de victorias de cada equipo durante la última temporada. Presenta valores entre 48.8 % y 74.4 %, con una media aproximada del 60.5 % y una mediana ligeramente menor (59.15 %). La desviación estándar (~ 8.51 pp) indica una dispersión moderada: existen diferencias apreciables entre equipos más y menos consistentes. Su curtosis (~ 1.69) sugiere una distribución con valores concentrados en un rango relativamente homogéneo. En términos competitivos, los equipos muestran variabilidad en desempeño, aunque sin extremos de dominio absoluto ni bajo rendimiento crítico.

**KDA** mide el equilibrio entre eliminaciones/asistencias y muertes. Se sitúa entre 1.54 y 4.40, con media de 3.30 y mediana de 3.44, lo que apunta a una distribución bastante simétrica. La desviación estándar (~ 0.88) refleja diferencias moderadas en eficiencia durante las partidas. La curtosis (~ 2.19) indica que la distribución es ligeramente platicúrtica, sin colas pesadas. En términos de juego, los equipos muestran distintos estilos y niveles de eficacia mecánica, pero sin valores extremadamente bajos ni excepcionalmente altos.

**Control_obj**, porcentaje de control de objetivos del mapa, presenta el rango más amplio del conjunto (30.1 %–78.4 %). La media (55.1 %) y la mediana (53.55 %) sugieren un desempeño medio-alto en la mayoría de equipos. Sin embargo, la desviación estándar (~ 14.85) evidencia alta heterogeneidad: algunos equipos dominan con claridad los objetivos, mientras que otros tienen dificultades para competir en esta faceta táctica. Su curtosis (~ 1.77) confirma la ausencia de colas marcadas y apunta a una distribución más aplanada que la normal.

Por último, **Entreno_horas**, las horas semanales de entrenamiento conjunto, oscila entre 18 y 40 h. La media (28.7 h) y la mediana (30.5 h) indican que la mayoría de equipos se sitúan alrededor de las 30 h semanales. La desviación estándar (~ 6.24 h) revela diferencias notables en la carga de trabajo, posiblemente asociadas a filosofías de entrenamiento o disponibilidad de recursos. Su curtosis (~ 2.22) muestra de nuevo una distribución relativamente plana sin valores extremos.

En conjunto, las variables presentan **variabilidad moderada y ausencia de valores extremos**, lo que indica un comportamiento estadístico estable en el que todos los equipos se sitúan en rangos razonables de rendimiento y dedicación. Esto proporciona una base adecuada para aplicar métodos de clustering, ya que las diferencias observadas pueden ayudar a identificar perfiles competitivos diferenciados sin verse distorsionados por outliers.


In [ ]:
# Seleccionar solo las variables numéricas
numeric_vars <- wdata[, sapply(wdata, is.numeric)]

# ---------- BOXPLOTS AUTOMÁTICOS ----------
par(mfrow = c(2, ceiling(ncol(numeric_vars)/2)))
for(i in 1:ncol(numeric_vars)){
  boxplot(numeric_vars[, i], main = colnames(numeric_vars)[i])
}

# ---------- HISTOGRAMAS AUTOMÁTICOS ----------
par(mfrow = c(2, ceiling(ncol(numeric_vars)/2)))
for(i in 1:ncol(numeric_vars)){
  hist(numeric_vars[, i], main = colnames(numeric_vars)[i], xlab = "")
}

# Volver a la configuración normal
par(mfrow = c(1,1))


**<u>Interpretación resultados</u>.–**

Los boxplots y los histogramas permiten comparar la **distribución**, **variabilidad** y **diferencias estructurales** entre las cuatro variables numéricas: *Win_rate*, *KDA*, *Control_obj* y *Entreno_horas*. A continuación se presenta un análisis comparativo conjunto.

**Comparación de dispersión y variabilidad**

A partir de los **boxplots**, se observa que ninguna variable presenta valores atípicos visibles, lo que indica estabilidad en las observaciones. Sin embargo, las variables difieren en su grado de dispersión:

- **Win_rate** y **Control_obj** son las métricas con **mayor variabilidad**. Los rangos son amplios y abarcan equipos con rendimientos bajos hasta equipos significativamente fuertes. Esto refleja heterogeneidad competitiva, tanto en victorias como en control de objetivos.
- **KDA** y **Entreno_horas** muestran **menor dispersión**, con valores más concentrados en intervalos estrechos. Esto sugiere comportamientos más homogéneos entre equipos en la eficiencia individual (KDA) y en el volumen semanal de entrenamiento.

**Comparación de la forma de las distribuciones**

Los **histogramas** permiten contrastar cómo se distribuye la información dentro de cada variable:

- **Win_rate** presenta una distribución relativamente uniforme, sin acumulaciones muy marcadas, lo que indica variedad equilibrada en rendimiento general.
- **KDA** muestra una ligera tendencia hacia valores medios-altos, lo que sugiere que la mayoría de equipos mantienen ratios de eficiencia razonablemente altos.
- **Control_obj** exhibe dos zonas destacadas: un grupo de equipos con control moderado y otro con valores altos, reflejando diferencias estratégicas en la toma de objetivos.
- **Entreno_horas** tiene una distribución algo sesgada hacia valores medios y altos, indicando que la mayoría de equipos entrenan entre 25 y 35 horas semanales, con pocos en los extremos.

 **Conclusión**

Las cuatro variables difieren significativamente en su estructura interna. Mientras que *Win_rate* y *Control_obj* presentan mayor heterogeneidad indicando diferencias claras en nivel competitivo y control estratégico del mapa variables como *KDA* y *Entreno_horas* son más homogéneas, lo que sugiere prácticas y niveles de eficiencia más similares entre los equipos. Esta comparación evidencia que la competitividad no está determinada solo por la práctica o la eficiencia individual, sino por diferencias estructurales más amplias entre equipos.


Para el análisis posterior es necesario normalizar las variables numéricas.

In [ ]:
# Seleccionar solo variables numéricas
num_vars <- wdata[, sapply(wdata, is.numeric)]

# Tipificar (escalado estándar)
datos <- as.data.frame(scale(num_vars, center = TRUE, scale = TRUE))

# Si quieres mantener también el nombre del municipio:
datos <- cbind(Equipo = wdata$Equipo,
                           datos)

datos


<div style="background-color:#5DADE2"><b>Apartado 2</b></div>

In [ ]:
# Obtenemos información gráfica sobre la relación de las variables
# Instalamos la librería GGally
install.packages("GGally")
install.packages("dplyr")

In [ ]:
# Cargamos la librería
library(GGally)
library(dplyr)

In [ ]:
# Seleccionar solo variables numéricas
datos_est <- datos %>% select(where(is.numeric))

# ggpairs solo con columnas numéricas
ggpairs(datos_est) +
  theme_minimal()


**<u>Interpretación resultados</u>.-**  
La matriz de gráficos permite analizar la estructura de relación entre las variables estandarizadas vinculadas al rendimiento y comportamiento del jugador: *Win_rate*, *KDA*, *Control_obj* y *Entreno_horas*. En términos generales, **las correlaciones son muy bajas**, lo que indica que las variables aportan información bastante independiente entre sí y, por tanto, pueden resultar útiles en procesos de clustering sin riesgo de redundancia excesiva.

En primer lugar, la relación entre **Entreno_horas y Win_rate** muestra la correlación más destacada (~ 0.21), aunque sigue siendo débil. Esta tendencia sugiere que **un mayor tiempo de entrenamiento podría asociarse ligeramente con un mayor porcentaje de victorias**, pero la dispersión observada revela que no existe un patrón lineal claro. Del mismo modo, la correlación entre **Entreno_horas y KDA** (~ 0.19) apunta a que un mayor volumen de práctica podría asociarse con un desempeño algo más eficiente en términos de bajas y asistencias, aunque de forma modesta.

Por otro lado, las variables **KDA** y **Win_rate** presentan una correlación prácticamente nula, lo cual indica que **tener buenos valores de KDA no garantiza necesariamente un mayor porcentaje de victorias**, probablemente por factores estratégicos o por el rol desempeñado dentro del equipo. De igual modo, **Control_obj** no muestra relaciones lineales evidentes con ninguna de las otras métricas (todas cercanas a 0), lo que sugiere que el dominio sobre objetivos del mapa depende de dinámicas distintas a las captadas por las otras variables.

En conjunto, la matriz indica que el conjunto de variables **no está fuertemente correlacionado**, lo cual es favorable para el análisis de clústeres, ya que cada una aporta una dimensión distinta del rendimiento o comportamiento del jugador. Esto permitirá que los métodos de agrupamiento (jerárquicos o no jerárquicos) detecten perfiles diferenciados basados en múltiples facetas del desempeño sin que ninguna variable domine excesivamente a las demás.


In [ ]:
# Función para obtener p-valores de cor.test
p_matrix <- function(df){
  vars <- colnames(df)
  n <- length(vars)
  M <- matrix(NA, n, n, dimnames = list(vars, vars))

  for(i in 1:n){
    for(j in 1:n){
      M[i,j] <- cor.test(df[[i]], df[[j]])$p.value
    }
  }
  return(M)
}

# Obtener matriz de p-valores
p_values_table <- p_matrix(datos_est)

# Mostrar la tabla
p_values_table

En un análisis de correlación, evaluamos si la relación observada entre dos variables podría explicarse simplemente por azar. Para ello se plantea un contraste de hipótesis:

- **Hipótesis nula ($H_0$)**:
No existe correlación real entre las dos variables.
Matemáticamente:

$$𝐻_0: 𝜌=0$$

- **Hipótesis alternativa ($H_1$)**:
Sí existe correlación entre las dos variables (puede ser positiva o negativa).
Matemáticamente:
$$𝐻_1: 𝜌≠0$$


El p-valor indica la probabilidad de obtener una correlación igual o más extrema que la observada, asumiendo que la hipótesis nula es cierta.

Un **p-valor pequeño (habitualmente p < 0.05**) sugiere que sería **muy improbable obtener esa correlación por azar**, por lo que se **rechaza $H_0$** y se concluye que existe una correlación significativa.

Un **p-valor grande** implica que los datos observados son compatibles con ausencia de correlación, por lo que **no se puede rechazar $H_0$**.

En este caso todos son valores ampliamente superiores a 0.05 lo que indica que sus correlaciones son nulas.

<div style="background-color:#5DADE2"><b>Apartado 3</b></div>

Ahora se calculará la matriz de proximidad o matriz de distancias entre las observaciones.

Se usará la función `dist()` para construir una matriz de distancias, que pertenece a la librería `base`. Se usará de la siguinete forma `dist(x, method = "euclidean")`, donde:

+ **x**: es el data numérico que contiene las observaciones y las variables.

+ **method**: indica el tipo de distancia que se desea calcular, en este caso se usará exclusivamnete:
  - "euclidean" (distancia euclidiana, por defecto)  

La función devolverá un objeto de clase `dist`, que almacena la matriz de distancias.




In [ ]:
# Calculamos la matriz de distancias empleando la distancia euclidiana
datos_dist = dist(datos_est, method = "euclidean")

In [ ]:
# Mostramos la estructura de la matriz de distancias
str(datos_dist)

In [ ]:
# Mostramos las primeras seis filas de la matriz de distancias
matrix_dist = as.matrix(datos_dist)[1:6, 1:6]
head(matrix_dist)

In [ ]:
# Instalamos la librería factoextra
install.packages("factoextra")

In [ ]:
# Cargamos la librería factoextra
library(factoextra)

In [ ]:
# Representamos mediante escalas de color la distancia entre todas las observaciones para ver
# si detectamos grupos de observaciones
suppressWarnings(
  fviz_dist(datos_dist, show_labels = TRUE)
)

**<u>Interpretación resultados</u>.-**   La distancia usada es la **distancia euclidiana** la cuantifica **cuán diferentes son dos observaciones** midiendo la longitud del segmento que las une en un espacio multidimensional.

Si cada observación viene dada por $p$ variables, la distancia entre dos individuos $x = (x_1,\dots,x_p)$ e $y = (y_1,\dots,y_p)$ se calcula como:

$$
d_{\text{euclidiana}}(x,y)
= \sqrt{(x_1 - y_1)^2 + \cdots + (x_p - y_p)^2 }.
$$

– **Cuanto menor es la distancia**, más parecidas son las observaciones.  
– **Cuanto mayor es la distancia**, más diferentes son.  

El mapa de distancias muestra la matriz de proximidad entre los 18 equipos tras tipificar las variables y calcular la **distancia euclidiana** entre cada par de ellos. Cada celda representa la distancia entre dos equipos:  
– Tonos **rojizos** → distancias bajas → equipos **muy similares**.  
– Tonos **azulados** → distancias altas → equipos **muy diferentes**.

La diagonal principal aparece en rojo intenso, como es natural, ya que la distancia de cada equipo consigo mismo es cero.

Al analizar el patrón global, aparecen **zonas rojizas bien definidas** (especialmente hacia el centro del mapa, en la zona superior izquierda y la inferior derecha), lo que indica **bloques de equipos muy parecidos entre sí**, probablemente compartiendo niveles similares de win rate, horas de entrenamiento, control sobre los objetivos y ratios bajas/asistencias. Estos bloques sugieren la presencia de **clústeres naturales y compactos**, que posteriormente deberían emerger con claridad en algoritmos no jerárquicos.

Por otro lado, se observan **manchas azul oscuro** localizadas, especialmente en la parte inferior derecha y también en ala parte superior izquierda. Estas zonas señalan equipos que son **marcadamente distintos** al resto, ya sea por su desempeño u horas de entrenamiento. Es probable que estos equipos actúen como **observaciones aisladas** dentro del clustering o formen grupos muy reducidos.

Finalmente, combinando las áreas rojizas continuas con las zonas azuladas dispersas, el mapa sugiere la existencia de **entre dos y cuatro grupos** de equipos con perfiles diferenciados, lo cual respalda la pertinencia de avanzar hacia un análisis de clúster no jerárquico.


<div style="background-color:#5DADE2"><b>Apartado 4</b></div>

A diferencia de los métodos jerárquicos, que construyen una secuencia completa de particiones anidadas, los enfoques no jerárquicos generan directamente una única partición del conjunto, lo que exige especificar o estimar parámetros clave como el número de grupos, la densidad mínima o la forma de los clusters. Esta característica los hace computacionalmente eficientes y particularmente adecuados para bases de datos de tamaño medio o grande.

En este estudio se considerarán **cuatro familias de métodos no jerárquicos**, cada una basada en un principio estructural diferente y capaz de capturar distintos tipos de patrones en los datos. Los **métodos basados en particiones** buscan la mejor asignación de observaciones a k clusters optimizando una función de distancia; los **métodos basados en densidad** detectan regiones densas y permiten descubrir grupos con formas arbitrarias; los **métodos basados en modelos** suponen que los datos provienen de una mezcla de distribuciones probabilísticas y asignan probabilidades de pertenencia a cada cluster; y los **métodos basados en grafos** utilizan relaciones de similitud para identificar comunidades en estructuras complejas que no necesariamente siguen métricas euclídeas.

Cada uno de estos enfoques será estimado e interpretado de manera individual, analizando cómo particionan los datos, qué tipo de estructura capturan y bajo qué supuestos operan. Este conjunto de métodos permite obtener una visión completa y comparada de la organización interna del conjunto de datos, destacando similitudes y diferencias entre estrategias basadas en geometría, densidad, probabilidad o conectividad.


El algoritmo **k-means** es uno de los métodos de *clustering* más utilizados debido a su eficiencia y simplicidad. Su objetivo es particionar un conjunto de datos en **k grupos no solapados**, de forma que cada observación pertenezca al cluster cuyo centro (o **centroide**) esté más próximo según una métrica de distancia,en este caso la **distancia euclidiana**.
El proceso es iterativo: primero asigna cada observación al centroide más cercano y después recalcula los centroides como la media de las observaciones asignadas. Este procedimiento se repite hasta que las asignaciones dejan de cambiar o se alcanza el número máximo de iteraciones. .

Su solución depende fuertemente de los valores iniciales de los centroides y puede converger a mínimos locales. Por ello es habitual ejecutar varias inicializaciones aleatorias y seleccionar la mejor solución.


Para estimar un modelo k-means en R se utiliza la función **`kmeans()`** de la librería *base*. Su sintaxis general es:

`kmeans(x, centers, iter.max = 10, nstart = 1, algorithm = "Hartigan-Wong")`

Los principales argumentos son:

- **`x`**: *data.frame* con **variables numéricas** sobre las que se realizará el clustering en este caso sería datos_est.
- **`centers`**: número de clusters **k** o bien un conjunto de centroides iniciales.
- **`nstart`**: número de distintas **inicializaciones aleatorias** a probar para evitar soluciones en mínimos locales ya comentada.
- **`iter.max`**: número máximo de iteraciones permitidas del algoritmo.
- **`algorithm`**: versión del algoritmo de optimización; por defecto `"Hartigan-Wong"` que es el más habitual y eficiente.



In [ ]:
# Fijamos semilla para reproducibilidad
set.seed(123)

In [ ]:
# Construímos el clúster no jerárquico con método k-means y distancia euclidiana
NHC_kmeans = kmeans(datos_est, centers = 4, nstart = 25, iter.max = 100, algorithm = "Hartigan-Wong")

In [ ]:
# Constatamos la clase del objeto
class(NHC_kmeans)

In [ ]:
# Información obtenida del clúster no jerárquico con método k-means y distancia euclidiana
str(NHC_kmeans)

El objeto contiene $9$ elementos los cuales se explicarán a continuación:

El elemento `cluster` es un vector de longitud igual al número de observaciones del conjunto de datos (18 en este caso). Cada posición contiene el **número de clúster asignado** a la observación correspondiente (el primer elemento estaría en el cluster número 4 junto al 7, 14 y 15). En k-means, esta asignación es el resultado final del algoritmo tras iterar entre asignar puntos al centroide más cercano y recalcular dichos centroides.


In [ ]:
# Número de clúster asignado a cada uno de los clientes
NHC_kmeans$cluster

El componente `centers` es una matriz de dimensión **k × p** donde `k` es el número de clústeres y `p` el número de variables empleadas. Cada fila contiene el **centroide** del clúster, expresado como la **media de cada variable dentro del grupo**.

En k-means el centroide tiene un significado completamente geométrico: es el **promedio de las observaciones asignadas** al clúster en el espacio estandarizado. Esto contrasta con el clustering jerárquico, donde la noción de centroide depende del método de enlace. Analizar estos valores permite comprender el "perfil medio" de cada grupo.


In [ ]:
# Información sobre los centroides de cada clúster
NHC_kmeans$centers

El elemento `totss` representa la **suma de cuadrados total** respecto a la media global del dataset. Es una medida de la **variabilidad total** de los datos antes de agrupar. Este valor no depende del número de clústeres `k` y sirve como referencia del total de variación presente.


In [ ]:
# Información sobre la suma de cuadrados total
NHC_kmeans$totss

Este componente es un vector de longitud **k**, donde cada elemento mide la **suma de cuadrados intra–clúster** del grupo correspondiente. Representa la **compactación interna** del clúster: valores bajos indican que las observaciones del grupo están muy próximas entre sí, mientras que valores altos reflejan alta dispersión.

En este caso el clusters 4 sería el más compacto y el 3 sería el que posee más dispersión.


In [ ]:
# Información sobre la suma de cuadrados intra-clúster de cada clúster
NHC_kmeans$withinss

`tot.withinss` es la suma total de todas las `withinss`. Cuantifica la **variabilidad que queda dentro de los clústeres** tras realizar la partición. Es precisamente el valor que se usa para construir el **método del codo**, comparando cómo disminuye esta magnitud cuando se aumenta el número de clústeres.

Una buena elección de `k` se refleja en valores relativamente bajos de `tot.withinss`, pero sin caer en aumentos exagerados de complejidad (añadir más clústeres de los necesarios).

In [ ]:
# Información sobre el total de la suma de cuadrados intra-clúster
NHC_kmeans$tot.withinss

El valor `betweenss` representa la **variabilidad explicada por la separación entre los clústeres**, es decir, por la distancia entre sus centroides. Teóricamente cumple que:

$$\text{totss} \approx \text{tot.withinss} + \text{betweenss}.$$

Cuanto mayor sea `betweenss` en proporción al total, mejor **separados** están los grupos y más estructura está captando el modelo. Por tanto, un clustering de buena calidad presenta `betweenss` alto y `tot.withinss` bajo.



In [ ]:
# Información sobre la suma de cuadrados entre cústeres
NHC_kmeans$betweenss

El componente `size` es un vector de longitud **k** que indica el **número de observaciones en cada clúster**. Revisar estos tamaños es crucial, ya que clústeres extremadamente pequeños pueden sugerir:

- presencia de **outliers** agrupados,  
- un clúster poco estable,  
- o una mala elección de `k`.

Clústeres más equilibrados suelen indicar una partición más robusta.

In [ ]:
# Información sobre el no. de observaciones en cada clúster
NHC_kmeans$size

El valor `iter` indica cuántas **iteraciones** necesitó el algoritmo k-means para converger. El algoritmo se detiene cuando las asignaciones no cambian o cuando se alcanza el máximo de iteraciones permitido.

Un número reducido de iteraciones suele implicar que los grupos han sido identificados con claridad desde el inicio. Por el contrario, muchas iteraciones pueden sugerir geometría más compleja o que los centroides iniciales estaban mal posicionados.

En este caso 2 ieraciones es un valor muy bajo.

In [ ]:
# Información sobre el no. de iteraciones del algoritmo
NHC_kmeans$iter

Finalmente, `ifault` es un **código de estado** que informa sobre posibles problemas durante la ejecución. El valor `0` implica que **la convergencia ha sido correcta**, sin errores numéricos como en este caso. Cualquier otro valor indicaría problemas como: falta de convergencia, centroides vacíos o errores en la inicialización.

Este componente permite garantizar que la solución obtenida es fiable antes de interpretarla.


In [ ]:
# Información sobre la convergencia del algoritmo
NHC_kmeans$ifault

El método **DBSCAN (Density-Based Spatial Clustering of Applications with Noise)** es uno de los algoritmos más utilizados para detectar **grupos de formas arbitrarias** y **manejar automáticamente el ruido**. A diferencia de K-means, no requiere fijar el número de clústeres y basa su funcionamiento en la idea de **densidad local** alrededor de cada punto. Esto lo convierte en una herramienta muy útil cuando los grupos no son esféricos o cuando existen observaciones atípicas.

El algoritmo parte de dos parámetros clave:  
- un **radio de vecindad** llamado $\epsilon$ (`eps`),  
- un **mínimo de puntos** dentro de ese radio denominado `minPts`.

A partir de estos parámetros, DBSCAN clasifica cada observación según la densidad de su entorno. El procedimiento general consta de los siguientes pasos:

1. **Para cada punto**, se calcula cuántos vecinos tiene dentro del radio $\epsilon$.  
2. **Si el número de vecinos ≥ minPts**, el punto se considera **núcleo** y se inicia (o expande) un clúster.  
3. A partir de cada punto núcleo, el algoritmo **"crece" el clúster** incorporando:
   - todos sus vecinos directos,
   - los vecinos de esos vecinos, siempre que sean también puntos núcleo.
4. Los puntos que no cumplen las condiciones de densidad se examinan:
   - si están cerca de un punto núcleo, se consideran **frontera**,  
   - si no pertenecen a ninguna región densa, se clasifican como **ruido**.
5. Se repite el proceso hasta revisar todos los puntos.

El resultado es una partición formada por **regiones densas** separadas por **zonas de baja densidad**, con la ventaja de que el método **no fuerza asignaciones**: permite dejar puntos sin clúster .



Como ya se ha mencionado el algoritmo distingue tres categorías de puntos:

**a) Puntos núcleo (core points)**
Son los puntos cuya **vecindad de radio $\epsilon$ contiene al menos `minPts`** observaciones (incluido el propio).  
Representan las regiones donde la densidad es suficientemente alta como para formar un clúster.

En términos geométricos:  
$$|N_\epsilon(x)| \geq \text{minPts}.$$

Estos puntos **generan y expanden los clústeres**.

**b) Puntos frontera (border points)**
Son puntos que **no cumplen** el criterio de densidad para ser núcleo, pero **están dentro del radio $\epsilon$ de un punto núcleo**.  
No generan nuevos clústeres, pero sí **se conectan a uno ya existente**.

Representan observaciones situadas en la periferia de los grupos densos.

**c) Puntos ruido (noise points u outliers)**
Son observaciones que **no cumplen ninguna de las dos condiciones anteriores**.  
No están suficientemente cerca de ningún punto núcleo y, por tanto, **no pertenecen a ningún clúster**.

Estos puntos son especialmente relevantes, ya que DBSCAN identifica de forma natural los **outliers estructurales** del dataset.



La función principal del paquete **`dbscan`** para implementar este método es: **`dbscan(x, eps, minPts, weights = NULL, borderPoints = TRUE, search = "kd", bucketSize = 10, splitRule = "suggest", ...)`** donde:



+ `x`: Matriz o data frame con las **variables numéricas estandarizadas** del clustering al igual que en kmeans no se introduce la matriz de distancias.

+ `eps`: Distancia máxima que define la **vecindad** de un punto.  
  Es el parámetro más importante: define qué tan "juntos" deben estar los puntos para considerarse parte de una misma región densa.


+ `minPts`:**Número mínimo de puntos** contandose a si mismo que debe haber en un entorno de radio `eps` para considerar que un punto es un **punto núcleo (core point)**. Con este parámetro se modera la densidad para formar un clúster.

+ `weights`: Vector de pesos para ponderar la densidad local. Si no se especifica, todos los puntos tienen peso $1$ como será en este caso.

+ `borderPoints`: Indica si los puntos frontera deben asignarse a un clúster (`TRUE`) o mantenerse como ruido (`FALSE`).


+ `search`: Tipo de estructura usada para búsquedas de vecinos. En este caso se usará  `"kd"` (la cual está por defecto) que utiliza un kd-tree, muy eficiente en dimensiones bajas/medias.

+ `bucketSize`: Controla el tamaño mínimo de los nodos del kd-tree, afectando al rendimiento. El valor por defecto es $10$, adecuado para este caso.

+ `splitRule`: Regla de división del kd-tree. No cambia el resultado, solo la eficiencia computacional por lo que se dejará por defecto.

En la función hay parámetros avanzados opcionales para búsquedas o matrices de distancias de ahílos puntos suspensivos en la sintaxis, al no utilizarse no se explicarán.



In [ ]:
# Instalación de la librería dbscan
install.packages("dbscan")

In [ ]:
# Cargamos la librería dbscan
library(dbscan)

In [ ]:
# Construímos el clúster no jerárquico con método dbscan
# eps: radio de vecindad
# minPts: mínimo número de puntos para formar una región densa
NHC_dbscan = dbscan(datos_est, eps = 1.5, minPts = 3)

In [ ]:
# Constatamos la clase del objeto
class(NHC_dbscan)

In [ ]:
# Información obtenida del clúster no jerárquico con bdscan
str(NHC_dbscan)

Los valores utilizados para el modelo se han variado debido a que si se redujerá la distancia o se aumentará el número de nodos necesario los clusters se reducen a un único cluster o identifica a todos como outliers.

El objeto contiene $5$ elementos los cuales se explicarán a continuación:

El elemento `cluster` es un vector de longitud igual al número de observaciones del conjunto de datos (18 en este caso). Cada posición contiene el **número de clúster asignado** a la observación correspondiente (el primer elemento estaría en el cluster número 1 junto al 2, 7, 11, 14 y 15). Los elementos que aparecen con un 0 significa que se han identificado como un outliers.


In [ ]:
# Número de clúster asignado a cada uno de los clientes
NHC_dbscan$cluster

El objeto **`eps`** indica el radio de vecindad $\epsilon$ usado por DBSCAN. Representa la **distancia máxima para que dos puntos se consideren vecinos**.

In [ ]:
# Información sobre el radio de vencidad utilizado
NHC_dbscan$eps

El objeto **`minPts`** indica el **número mínimo de puntos necesarios** usado por DBSCAN.

In [ ]:
# Información sobre el no. mínimo de puntos necesarios par considerar un clúster
NHC_dbscan$minPts

Este objeto  **`metric`** indica la **métrica empleada para calcular distancias** entre observaciones usada por DBSCAN.

In [ ]:
# Información sobre la métrica utilizada
NHC_dbscan$metric

Este objeto **`borderPoints`** indica si los puntos frontera se incluyen en los clústeres. En este caso se usao el por defecto `TRUE` lo que significa es que los puntos frontera se asignan al clúster correspondiente.

In [ ]:
# Información sobre si se consideran los puntos frontera o no
NHC_dbscan$borderPoints

El método **Gaussian Mixture Models (GMM)** es un enfoque probabilístico para el clustering donde se asume que los datos proceden de una **mezcla de distribuciones gaussianas multivariantes**, cada una representando un clúster. A diferencia de k-means (que asigna cada punto a un único clúster), GMM asigna **probabilidades de pertenencia**, permitiendo capturar clústeres con **formas elipsoidales**, **distintas varianzas** y **correlaciones internas** entre variables.

El procedimiento general consta de los siguinetes pasos:
1. Se generan estimaciones iniciales de medias (vector centroide), matrices de covarianza y pesos de mezcla. Si no se inicializan se hará una asignación aleatoria.

2. Antes de asignar una observación a un cluster a cada observación el algoritmo calcula la **probabilidad posterior de pertenecer a cada cluster**, es decir, las **responsabilidades**:  
  $$\tau_{ik} = P(\text{corresponder al clúster } k \mid x_i).$$
   Esto permite modelar incertidumbre: un equipo puede tener un $70 \%$ de pertenencia al clúster 1 y un $30 \%$ al clúster 3 o tener un $52 \%$ y un $48 \%$ en cuyo caso no estría tan claro a cual pretenece.
  
3. A partir de las responsabilidades, se actualizan los parámetros del modelo: medias, matrices de covarianza y pesos. Y con cada iteración se mejora la verosimilitud del modelo.

4. Los pasos anteriores se repiten hasta que la log-verosimilitud se estabiliza o se alcanza el número máximo de iteraciones. Esta naturaleza iterativa hace que GMM sea más flexible que métodos como DBSCAN y más expresivo que k-means.

5. Tras relizar todos lo pasos previos la observación se asocia al cluster con mayor probabilidad.




La función principal para estimar un GMM en R es: **`Mclust(data, G = NULL, modelNames = NULL, prior = NULL, control = emControl(), initialization = NULL, ...)`**. A continuación se explican sus argumentos más importantes:

- **`data`**: matriz o data.frame con las variables numéricas. Al igual que los métodos anteriores no utiliza matriz de distancias.

- **`G`**: conjunto de números de componentes (clusters) a evaluar. Si se deja por defecto, se usa automáticamente `1:9` lo que significa que se hará con $G=1$ hasta $G=9$. El modelo óptimo se selecciona por BIC.

- **`modelNames`**: define la estructura de las matrices de covarianza (en forma: esféricas, diagonales, elipsoidales o en volumen). Si se deja `NULL`, se prueban modelos adecuados según la dimensión del dataset y también se elige el mejor según BIC.

- **`prior`**: especifica un prior conjugado para medias y covarianzas mediante `priorControl()`. Se usará su valor por defecto `NULL` (estimación por máxima verosimilitud sin información previa).

- **`control`**: parámetros del algoritmo EM (tolerancias, iteraciones máximas), definidos con `emControl()`. Permitiendo un mayor ajuste sobre el modelo.



- **`initialization`**: controla la inicialización del EM. Puede incluir:
  - `hcPairs` (partición jerárquica previa),
  - `subset` (subconjunto para inicializar),
  - `noise` (componente adicional para ruido).
  Si no se especifica, `Mclust()` realiza automáticamente una inicialización jerárquica robusta.

En la función hay parámetros avanzados opcionales para la representación de gráficas de ahí los puntos suspensivos en la sintaxis, al no utilizarse no se explicarán.




In [ ]:
# Instalación de la librería mclust
install.packages("mclust")

In [ ]:
# Cargamos la librería mclust
library(mclust)

In [ ]:
# Construímos el clúster no jerárquico con modelos de mezclas gaussianas
# data: base de datos estandarizada (datos_est)
# G = 2:4 orquilla de 2 a 4 componentes (clústeres) para facilitar la comparación con k-means y DBSCAN
NHC_gmm = Mclust(data = datos_est, G = 2:4)

In [ ]:
# Constatamos la clase del objeto
class(NHC_gmm)

In [ ]:
# Información obtenida del clúster no jerárquico con modelos de mezclas gaussianas
str(NHC_gmm)

El objeto contiene 16  elementos los cuales se explicarán a continuación:

+ **`cluster`**: Muestra la **instrucción exacta utilizada para estimar el modelo**. Igual que en clustering jerárquico (`call` en `hclust`) o en k-means, sirve como referencia documental.

In [ ]:
# Instrucción exacta utilizada para estimar el modelo GMM
NHC_gmm$call

+ **`data`**: Contiene **los datos estandarizados utilizados en el modelo**.  
Es una matriz numérica de dimensión **18×4** (18 observaciones, 4 variables).  
Incluye nombres de las variables:  
`Win_rate`, `KDA`, `Control_obj`, `Entreno_horas`.

In [ ]:
# Matriz de datos utilizada (660 observaciones x 5 variables estandarizadas)
head(NHC_gmm$data)

+ **`modelName`**: Indica el **modelo de la matriz de covarianzas utilizado**. En este caso `"VII"` es el **tipo de estructura de covarianza elegido por BIC** entre todas las posibles.  
En este caso:  
- V = volumen variable  
- I = forma esférica  
- I = orientación no aplicable (esférico)

Modelo **esférico con varianzas diferentes entre grupos** el resto de posisbilidades están detallasdas más abajo.

In [ ]:
# Tipo de modelo gaussiano elegido (VVV = covarianzas completamente libres)
NHC_gmm$modelName

+ **`n`**: Indica el **número de observaciones** consideradas en el dataset.

In [ ]:
# Número total de observaciones
NHC_gmm$n

+ **`d`**: Indica el **número de variables** consideradas en el dataset.

In [ ]:
# Número de variables (dimensión del espacio)
NHC_gmm$d

+ **`G`**: Indica el **número de componentes (clústeres)** seleccionados por el algoritmo EM (Expectation Maximization) entre los propuestos.

In [ ]:
# Número de componentes gaussianos seleccionados
NHC_gmm$G

+ **`BIC`**: contiene **los valores del BIC para todos los modelos probados**.  
Es una matriz con:
- filas: componentes evaluadas (`2`, `3`, `4`)
- columnas: tipos de modelos gaussianos (`EII`, `VII`, `EEI`, …)

Se usa para seleccionar el mejor modelo.

En la celda de código se explican todas las posibles combinaciones de los modelos gaussianos

In [ ]:
# Matriz de valores BIC para los diferentes modelos gaussianos evaluados
# 1º letra (Volumen): Tamaño del clúster
# 2º letra (Forma): Relación entre las longitudes de los ejes principales - elipticidad
# 3º letra (Orientación): Rotación de la elipse en el espacio - matriz de eigenvectores

# EII: Volumen igual, forma esférica, sin orientación -> Clústeres esféricos, del mismo tamaño, sin rotación. Es el modelo equivalente a k-means
# VII: Volumen distinto, forma esférica, sin orientación -> Clústeres esféricos pero con tamaños diferentes.
# EEI: Volumen igual, forma igual, sin orientación -> Elipses con misma forma pero no rotadas; sólo difieren en la ubicación de su centro.
# VEI: Volumen distinto, misma forma, sin orientación -> Mismas elipses sin rotación, pero unas más grandes que otras.
# EVI: Volumen igual, forma distinta, sin orientación -> Todas las elipses tienen mismo volumen pero diferente elipticidad.
# VVI: Volumen distinto, forma distinta, sin orientación -> Elipses sin rotación pero cada clúster tiene volumen y forma distinta.
# EEE: Volumen igual, forma igual, orientación igual -> Misma elipse (misma covarianza) en todos los clústeres.
# VEE: Volumen distinto, forma igual, orientación igual -> Clústeres con estructura idéntica pero tamaños distintos.
# EVE: Volumen igual, forma distinta, orientación igual -> Diferente elipticidad pero misma rotación.
# VVE: Volumen distinto, forma distinta, orientación igual -> Diferencias en tamaño y forma, pero mismos ejes de orientación.
# EEV: Volumen igual, forma igual, orientación distinta -> Todas las elipses tienen tamaño y forma igual pero están rotadas de distinta forma.
# VEV: Volumen distinto, forma igual, orientación distinta -> Volúmenes diferentes, misma forma, orientaciones distintas.
# EVV: Volumen igual, forma distinta, orientación distinta -> Mismo volumen pero formas y orientaciones diferentes.
# VVV: Volumen distinto, forma distinta, orientación distinta -> El modelo más flexible de todos, sin restricciones.
NHC_gmm$BIC

+ **`loglik`**: Valor de la **log-verosimilitud final** del modelo. Cuanto mayor sea, mejor ajusta el modelo.

In [ ]:
# Log-verosimilitud del modelo GMM estimado
NHC_gmm$loglik

+ **`df`**: Grados de libertad del modelo (**número de parámetros estimados**). Dependen de las medias, pesos y covarianzas a estimar.

In [ ]:
# Grados de libertad (parámetros estimados del modelo)
NHC_gmm$df

+ **`bic`**: Valor del BIC final del modelo elegido (el mayor entre todas las combinaciones probadas)

In [ ]:
# Valor del BIC del modelo finalmente elegido por Mclust
NHC_gmm$bic

+ **`icl`**: Valor del ICL (Integrated Completed Likelihood), penaliza el solapamiento entre clusters.  
Valor cercano al BIC indica que los clústeres están bien separados. Como se puede apreciar la diferencia es inferior a una unidad inferior al $1 \%$ del Bic.

In [ ]:
# Valor del índice ICL (penaliza incertidumbre de clasificación)
NHC_gmm$icl

+ **`hypvol`**: Volumen del hiperparalelogramo para modelos 1D.  
En este caso es `NA` porque el modelo es multivariante.

In [ ]:
# Volumen hiperdimensional del espacio de parámetros (habitualmente NA)
NHC_gmm$hypvol

+ **`parameters`**:Lista con los parámetros del modelo GMM seleccionado.

a) `pro`
Pesos de mezcla: $0.85\%$ y $0,15\%$ lo que siginifica que el **clúster 1 contiene el 85.3%** de los jugadores; el **clúster 2 el 14.7%**.

b)`mean`
Matriz de medias (**4 variables × 2 componentes**), una columna por clúster.  
En este caso:
- Clúster 1: jugadores con Win_rate positivo.  
- Clúster 2: jugadores con métricas notablemente menores.

c) `variance`
Estructura de covarianza donde se pueden extraer múltiples datos, pero solo se extraerá:

- `sigma`: matrices de covarianza completas (4×4×2).  
  Cada componente produce una matriz distinta.



In [ ]:
# Proporciones de mezcla (probabilidad a priori de cada clúster)
NHC_gmm$parameters$pro

In [ ]:
# Medias (centroides gaussianos) por clúster y variable
NHC_gmm$parameters$mean

In [ ]:
# Matrices de covarianza para cada componente (definen forma y orientación)
str(NHC_gmm$parameters$variance$sigma)

In [ ]:
# Extraer la matriz de covarianza del clúster 1
cov_1 = NHC_gmm$parameters$variance$sigma[,,1]
cov_1

In [ ]:
# Extraer la matriz de covarianza del clúster 2
cov_2 = NHC_gmm$parameters$variance$sigma[,,2]
cov_2

+ **`z`**: Matriz de probabilidades de pertenencia (**responsabilidades EM**).  
Con una dimensión: **18×2** ya que son 18 equipos y solo dos clusters. Los valores oscilan entre $0$ y $1$. En este caso la primera observación $1.00$, $0.00$ indica que el equipo es prácticamente seguro que se encuentra en el clúster $1$.

In [ ]:
# Probabilidades de pertenencia de cada observación a cada clúster
NHC_gmm$z[1:5, ]

+ **`classification`**: Vector con la **asignación final** de cada observación siguinedo la regla del máximo a posteriori. En este caso el quipo número 4, 6 y 12 pertenecerían al segundo cluster y el resto pertenecerían al primero.

In [ ]:
# Clasificación final de cada cliente (clúster con mayor probabilidad)
NHC_gmm$classification

+ **`uncertainty`**: Mide la **incertidumbre de clasificación** siguiendo la siguinete ecuación:  
$$1 - \max_k(\tau_{ik})$$
Valores cercanos a **0** indican que la clasificación muy segura.  
Mientras que los valores altos indican la existencia de puntos en frontera entre clústeres.

En este caso como se puede observar serían muy proximos a 0 lo que refuerza la división realizada.

In [ ]:
# Incertidumbre asociada a la clasificación de cada observación
head(NHC_gmm$uncertainty, 10)

El método **Spectral Clustering (SPEC)** es un enfoque basado en grafos que realiza el clustering analizando la **estructura de conectividad** entre las observaciones, en lugar de asumir distribuciones estadísticas como en GMM. El algoritmo construye una matriz de afinidad que mide la similitud entre pares de puntos y, a partir de ella, obtiene **la matriz Laplaciana**. Estos autovectores transforman los datos a un espacio vectorial donde los clústeres se vuelven linealmente separables, permitiendo aplicar k-means de forma eficaz. A diferencia de métodos basados en centroides o distribución, SPEC puede detectar clústeres no convexos, estructuras complejas y relaciones que dependen de la geometría del grafo, sin suponer gaussianidad ni formas elipsoidales.

El procedimiento general consta de los siguinetes pasos:

- **1. Construcción del modelo a partir de un grafo**
Spectral Clustering comienza interpretando los datos como un **grafo de similitud**:
  - Cada observación es un **nodo**.
  - La similitud entre observaciones define una **arista** del grafo.
  - El usuario debe fijar el número de clústeres $K$.

- **2. Construcción de la matriz de afinidad, grado y Laplaciana**
El algoritmo construye internamente:

  - **Matriz de afinidad** $W$: usando un kernel (típicamente RBF o KNN), calcula cuánta similitud hay entre pares de puntos.
  - **Matriz de grado** $D$: contiene la suma de las afinidades de cada nodo.
  - **Matriz laplaciana** $L = D - W$: describe la estructura del grafo y será la base del análisis espectral.
- **3. Descomposición espectral**
Se realiza una descomposición en autovalores y autovectores:

  - Los **autovalores más pequeños** contienen la estructura natural del grafo.
  - Se descarta el **primer autovector** $\mathbf{v}_1$ ya que está asociado al autovalor 0.
  - Se eligen los **$K$ autovectores siguientes**: $\mathbf{v}_2, \dots, \mathbf{v}_{K+1}$.
  - Estos autovectores forman la **matriz espectral** $U$, que reubica los datos en un espacio donde los clústeres son linealmente separables.

- **4. Clustering en el espacio espectral**
  En este nuevo espacio (filas de $U$), ya no se trabaja con las coordenadas originales:
  - Se aplica **k-means** sobre las filas de $U$.
  - k-means funciona correctamente aquí porque los clústeres ya están separados de forma casi lineal.

- **5. Asignación final de clústeres**
  El algoritmo asigna directamente sin hacer iteracions una etiqueta por punto:
  - No existen puntos “border” o “core” (DBSCAN).
  - No existen probabilidades ni incertidumbres (GMM).
  - No hay ruido explícito.
  - Los clústeres pueden tener **formas no convexas**, algo que GMM y k-means no pueden capturar en el espacio original.






La función principal es `specc()`,de la librería $\tt kernlab$. Cuya sintaxis es **`specc(x, centers, kernel = "rbfdot", kpar = "automatic", iterations = 200, na.action = na.omit, ...)`** donde:

+ `x`: Matriz o data.frame con datos numéricos estandarizados. A partir de ellos se construye automáticamente la matriz de afinidad y el grafo de similitud.

+ `centers`: Número de clústeres $K$ o matriz de centros iniciales en el espacio espectral. En este caso, se especificará  `centers = 2`.

+ `kernel`: especifica el tipo de kernel usado para calcular la afinidad.En esta práctica se usará kernel = "rbfdot" → kernel Gaussiano (RBF). Aunque habría otras obciones como "laplacedot", "polydot", "vanilladot" entre otros.

+ `kpar`: controla los **parámetros del kernel**. Puede ser una cadena de texto, como `"automatic"`, que elige valores adecuados automáticamente, o una lista, pkpar = list(sigma = 0.05). Aunque en este caso se usará el por defecto.

+ `iterations`: número **máximo de iteraciones internas** del algoritmo k-means. Por defecto suele ser `iterations = 200`.

+ `na.action`: indica **cómo tratar los valores perdidos** (`NA`). Por defecto se eliminan utilizando `na.action = na.omit`.

En la función hay parámetros avanzados opcionales para la normalización del grafo, ajustes de k-means u opciones del kernel de ahí los puntos suspensivos en la sintaxis, al no utilizarse no se explicarán.

In [ ]:
# Instalación de la librería kernlab
install.packages("kernlab")

In [ ]:
# Cargamos la librería kernlab
library(kernlab)

In [ ]:
# Ayuda de la función specc(x, centers, kernel, kpar, ...)
?specc

In [ ]:
# Construímos el clúster no jerárquico con Spectral Clustering
# x: base de datos estandarizada (datos_est)
# centers : fijamos 2 clústeres para facilitar la comparación con k-means, DBSCAN y GMM
NHC_spec = specc(x = as.matrix(datos_est), centers = 2, kernel  = "rbfdot")

In [ ]:
# Constatamos la clase del objeto
class(NHC_spec)

In [ ]:
# Información obtenida del clúster no jerárquico con Spectral Clustering
str(NHC_spec)

El objeto contiene 5 elementos los cuales se explicarán a continuación:

+ **`Data`**: Es un vector de enteros que contiene la asignación final de clústeres para cada observación.

En este caso habría 9 equipos en cada cluster.

In [ ]:
# Vector con la asignación final de clúster para cada observación (1 a 660)
NHC_spec@.Data

+ **`centers`**: Indica la información sobre los **centroides en el espacio transformado**. Aunque el clustering se hace en el espacio espectral (autovectores del Laplaciano del grafo), `specc()` devuelve los **centroides reconstruidos en el espacio original** para facilitar la interpretación. No son los centroides reales del clustering, sino una aproximación útil para interpretar perfiles medios de cada clúster.

Hay dos filas ya que solo se ha dividido en dos clusters, en este caso da la casualidad de que están exactamente en el extremo opuesto $C_1 = -C_2$.

In [ ]:
# Matriz de 2 x 5 con los centroides de los clústeres en el espacio original
NHC_spec@centers

+ **`size`**: Es un vector que indica el número de puntos que pertenecen a cada clúster. Como ya se ha comentado estna en euilibrio 9 en cada cluster.

In [ ]:
# Número de observaciones asignadas a cada clúster
NHC_spec@size

+ **`kernelf`**: Es un objeto que define el kernel utilizado para construir la matriz de afinidad del grafo en este caso representa al kernel gaussiano $K(x, y) = \exp\!\left(-\frac{\|x - y\|^{2}}{2\sigma^{2}}\right)$. El valor del parámetro `sigma` controla la escala de afinidad entre puntos. Un **sigma alto** indica afinidad suave (más conexiones). En cambio, un **sigma bajo** es sinónimo de afinidad muy localizada (grupos más pequeños y definidos). En este caso sería pequño por lo que la afinidad está muy localizada.

In [ ]:
# Parámetro sigma del kernel RBF utilizado para construir la matriz de afinidad
NHC_spec@kernelf@kpar$sigma

+ **`withinss`**: Es un vector numérico que contiene la Suma de cuadrados dentro de clústeres (WSS) pero en el espacio espectral, no en el espacio original. **Midiendo la cohesión interna** de cada clúster.

In [ ]:
# Suma de cuadrados dentro de cada clúster en el espacio original
NHC_spec@withinss

<div style="background-color:#5DADE2"><b>Apartado 5</b></div>

En esta sección se determinará el **número óptimo de clústers** para los modelos de clustering no jerárquico estimados previamente. El objetivo es encontrar el método de asociación y el número de clusters par ala base de datos.

Para cada una de estos métodos se aplicarán **tres criterios distintos de selección de clústers**, con una función propia ya que no esta implementada en `R`, con el fin de obtener una recomendación robusta y comparativa:

1. **Método del codo (traceW)**: analiza la disminución de la suma de inercias intra-clúster (suma de las distancias al cuadrado de cada una de las observaciones al respectivo centro del clúster) a medida que aumenta el número de grupos. El número óptimo de clústers se identifica en el punto donde la reducción deja de ser significativa. Siendo el número óptimo de clústeres q el que poseea al valor de k para el cuál se presente una caída repentina y más grande en el WSS.

2. **Estadístico de Gap**: compara la compactación observada con la que cabría esperar en configuraciones aleatorias, seleccionando el número de clústers para el cual la separación entre ambas es máxima.

3. **Silueta promedio**: evalúa simultáneamente la cohesión interna y la separación entre clústers; se elige el número de grupos que maximiza la calidad global de la partición. La silueta para una observación determinada $i$ es una <u>**combinación de la cohesión</u> $a_i$ y $b_i$** definida como: $s_i = \frac{b_i - a_i}{\max\{a_i,\, b_i\}}$. Esto es equivalente a:

\begin{cases}
1 - \dfrac{a_i}{b_i}, & \text{si } a_i < b_i,\\[6pt]
0, & \text{si } a_i = b_i,\\[6pt]
\dfrac{b_i}{a_i} - 1, & \text{si } a_i > b_i.
\end{cases}




Ya que la función es propia se dará una explicación de lo que hace.
La función `NHC_elbow` constituye una herramienta integrada para aplicar el método del codo en varios algoritmos de clustering no jerárquico: **k-means, Gaussian Mixture Models (GMM), Spectral Clustering y DBSCAN**. Su estructura está diseñada para garantizar reproducibilidad, control del rango de búsqueda y comparabilidad entre métodos. Para ello, la función comienza verificando y cargando automáticamente las librerías necesarias, transformando los datos en formato matricial y fijando una semilla. Posteriormente define una función auxiliar que calcula la *Within-Cluster Sum of Squares (WSS)*, magnitud clave para el método del codo en todos los algoritmos basados en
k.

El procedimiento para cada método se adapta a sus particularidades. En **k-means**, la función emplea `NbClust` para obtener la WSS clásica (tracew) y sugerir un número óptimo de clústeres. En **GMM** y **Spectral Clustering**, donde la formulación no incluye directamente WSS, se calcula una WSS aproximada a partir de las particiones asignadas a cada valor de k. En el caso de **DBSCAN**, donde no existe parámetro k, se estima el "codo" analizando la distancia al minPts-ésimo vecino más cercano y evaluando cómo varía el número de clústeres y el ruido para una rejilla de valores de `eps`.

Finalmente, la función organiza todos los resultados en una lista estructurada y genera **dos tablas comparativas**: una para los métodos basados en k y otra para DBSCAN. Estas tablas permiten observar de forma conjunta cómo evoluciona la WSS o el número de clústeres según el parámetro estudiado, facilitando la selección del modelo más adecuado. Esta integración en un único flujo de trabajo permite evaluar cuatro algoritmos no jerárquicos mediante un marco común y sistemático, lo que mejora la interpretación y la toma de decisiones en el análisis de clústeres.

In [ ]:
#--------------------------------------------------------------
# Función general para aplicar el "método del codo" a:
#  - k-means (usando NbClust y tracew)
#  - GMM (Mclust, WSS aproximada)
#  - Spectral Clustering (specc, WSS aproximada)
#  - DBSCAN (kNN distance para elegir eps)
#--------------------------------------------------------------
NHC_elbow = function(data, k_min = 2, k_max = 10, eps_grid = seq(0.4, 1.2, by = 0.05), minPts_dbscan = 3, seed = 123){

#----------------------------------------------------------
# 0) Comprobaciones y carga de librerías
#----------------------------------------------------------

# Aseguramos que 'data' sea una matriz numérica
data = as.matrix(data)

# Vector de valores de k a evaluar
k_values = k_min:k_max

# Establecemos semilla de reproducibilidad
set.seed(seed)

libs_necesarias = c("NbClust", "mclust", "kernlab", "dbscan", "cluster", "knitr")

for (lib in libs_necesarias) {
  if (!requireNamespace(lib, quietly = TRUE)) {
    install.packages(lib)
  }
  library(lib, character.only = TRUE)
}

# Cargamos librerías necesarias
library(NbClust)   # Para k-means + tracew
library(mclust)    # Para GMM
library(kernlab)   # Para Spectral Clustering
library(dbscan)    # Para DBSCAN
library(cluster)   # Para funciones de distancia
library(knitr)     # Para realizar la tabla comparativa

#----------------------------------------------------------
# 1) Función auxiliar: calcula WSS dada una partición
#     x  : matriz de datos (n x p)
#     cl : vector de clústeres (longitud n)
#----------------------------------------------------------
wss_clusters = function(x, cl) {

    # Para cada clúster j calculamos la suma de cuadrados
    grupos = split.data.frame(x, cl)

    # Eliminamos posibles grupos vacíos (por seguridad)
    grupos = grupos[sapply(grupos, nrow) > 0]

    # Aplica una función a cada clúster de la lista 'grupos'
    wss_vec = vapply(grupos, function(g) {
      centro = colMeans(g) # Calcula el centroide del clúster (media por columnas)
      sum(rowSums((g - centro)^2)) # Calcula la WSS del clúster: suma de distancias cuadráticas de cada punto al centroide
    }, numeric(1)) # Especifica que cada iteración devuelve un único número (WSS de un clúster)

    # Suma las WSS de todos los clústeres para obtener la WSS total
    sum(wss_vec)
}

#----------------------------------------------------------
# 2) K-MEANS + NBCLUST (método del codo "clásico" con tracew)
#----------------------------------------------------------
wss_km = NbClust(data = data, distance = "euclidean", min.nc = k_min, max.nc = k_max, method = "kmeans", index = "tracew")

# 'All.index' contiene el índice tracew (WSS) para cada k
tracew_km = wss_km$All.index

# Extraemos el número óptimo de clústeres según NbClust
best_km = as.numeric(wss_km$Best.nc[1])

#----------------------------------------------------------
# 3) GMM (Mclust) - WSS aproximada para cada G = k
#----------------------------------------------------------
wss_gmm = numeric(length(k_values))

for (i in seq_along(k_values)) {
  k = k_values[i]

  # Ajustamos el modelo de mezclas gaussianas con G = k
  gmm_k = Mclust(data = data, G = k)

  # Clasificación máxima a posteriori (partición dura)
  cl = gmm_k$classification

  # Calculamos WSS "tipo k-means" a partir de la partición
  wss_gmm[i] = wss_clusters(data, cl)
}

#----------------------------------------------------------
# 4) SPECTRAL CLUSTERING - WSS aproximada para cada k
#----------------------------------------------------------
wss_spectral = numeric(length(k_values))

for (i in seq_along(k_values)) {
  k = k_values[i]

  # Ajustamos Spectral Clustering con k clústeres
  spec_k = specc(x = data, centers = k, kernel = "rbfdot")

  # Asignación de clústeres (vector de longitud n)
  cl = spec_k@.Data

  # Calculamos WSS "tipo k-means"
  wss_spectral[i] = wss_clusters(data, cl)
}

#----------------------------------------------------------
# 5) DBSCAN - "codo" de distancias k-NN para eps
#----------------------------------------------------------
# Distancias al minPts-ésimo vecino más cercano (para kNNdistplot)
kNN_dists = kNNdist(data, k = minPts_dbscan)
kNN_dists = sort(kNN_dists)  # ordenamos para visualizar el "codo"

# Evaluamos una rejilla de eps para ver número de clústeres y ruido
n_clusters_db = numeric(length(eps_grid))
n_noise_db    = numeric(length(eps_grid))

for (i in seq_along(eps_grid)) {
  eps_i = eps_grid[i]

  # Ajustamos DBSCAN para cada eps de la rejilla
  db_i = dbscan(data, eps = eps_i, minPts = minPts_dbscan)

  # Número de clústeres distintos de 0 (excluimos ruido)
  n_clusters_db[i] = length(setdiff(unique(db_i$cluster), 0))

  # Número de puntos etiquetados como ruido (cluster = 0)
  n_noise_db[i] = sum(db_i$cluster == 0)
}

#----------------------------------------------------------
# 6) Construimos tablas resumen comparativas
#----------------------------------------------------------

# Tabla comparativa para métodos basados en k (k-means, GMM, Spectral)
# Intentamos alinear tracew_km con k_values usando los nombres (por si NbClust devuelve nombres de k)
tracew_km_num = as.numeric(tracew_km[match(k_values, names(tracew_km))])

tabla_k = data.frame(
  k                   = k_values,       # Número de clústeres
  tracew_kmeans       = tracew_km_num,  # Suma de cuadrados intra-clúster según NbClust (k-means)
  wss_gmm             = wss_gmm,        # WSS aproximada para GMM
  wss_spectral        = wss_spectral    # WSS aproximada para Spectral Clustering
)

# Tabla comparativa para DBSCAN en función de eps
tabla_dbscan = data.frame(
  eps        = eps_grid,      # Valores de eps evaluados
  n_clusters = n_clusters_db, # Número de clústeres (excluyendo ruido) para cada eps
  n_noise    = n_noise_db     # Número de puntos ruido para cada eps
)

#----------------------------------------------------------
# 7) Devolvemos resultados en una lista estructurada
#----------------------------------------------------------
resultados = list(
  kmeans = list(
    nbclust_obj = wss_km,        # objeto completo de NbClust
    tracew      = tracew_km,    # índice tracew para cada k
    best_k      = best_km     # número óptimo de clústeres según NbClust
  ),
  gmm = list(
    k_values = k_values,        # valores de k evaluados
    wss      = wss_gmm          # WSS aproximada para cada k
  ),
  spectral = list(
    k_values = k_values,        # valores de k evaluados
    wss      = wss_spectral     # WSS aproximada para cada k
  ),
  dbscan = list(
    minPts     = minPts_dbscan, # minPts utilizado en DBSCAN
    kNN_dists  = kNN_dists,     # distancias k-NN ordenadas (para el "codo" de eps)
    eps_grid   = eps_grid,      # rejilla de eps evaluada
    n_clusters = n_clusters_db, # número de clústeres (excluyendo ruido) para cada eps
    n_noise    = n_noise_db     # número de puntos ruido para cada eps
  ),
  tablas = list(
    k_methods = tabla_k,        # tabla comparativa k-means, GMM, Spectral
    dbscan    = tabla_dbscan   # tabla comparativa para DBSCA
    )
  )


# Devuelve el resultado final
return(resultados)
}


In [ ]:
# Aplicamos la función NHC_elbow sobre nuestra base de datos estandarizada
res_elbow = NHC_elbow(datos_est)

In [ ]:
# Información obtenida con la función 'NHC_elbow' (vista completa)
show(res_elbow)

**Interpretación.-** El objeto resultante contiene  5 elemnetos que resumen los resultados de los cuatro métodos de *clustering* no jerárquico aplicados mediante la función `NHC_elbow`. En primer lugar, dentro del apartado **`kmeans`**, observamos los valores del índice **tracew** para $k = 2,\dots,10$ (`nbclust_obj$All.index`), donde la inercia intra–clúster disminuye de forma progresiva. El método `NbClust` selecciona **$k = 4$** como número óptimo de clústeres (`best_k`), también se le puede añadir el valor correspondiente a ese número óptimo (`nbclust_obj$Best.nc`), proporcionando además la partición completa de las observaciones (`nbclust_obj$Best.partition`). Esta coherencia entre la caída pronunciada de tracew y la elección automática indica que el método encuentra una estructura relativamente clara para cuatro grupos.

En cuanto al apartado **`gmm`**, se muestran los valores de la WSS aproximada (`wss`)para cada $k$ (`k_values`) bajo modelos de mezclas gaussianas. En este caso, los valores no siguen un patrón estrictamente decreciente porque la WSS se calcula a partir de una clasificación dura derivada del modelo probabilístico. Aun así, se observa cierta estabilidad en torno a $k = 2,3$ y $k = 10$, mientras que otros valores presentan incrementos debidos a ajustes de mezcla más complejos.


En **`spectral`**, la WSS aproximada (`wss`) muestra un comportamiento similar: cierta variabilidad en función de $k$, con valores relativamente moderados entre $k=2$ y $k=10$ (`k_values`), aunque sin un "codo" claro. Esto indica que, para estos datos, ni GMM ni Spectral Clustering muestran una estructura de clústeres tan marcada como la identificada por k-means.

Finalmente, el apartado **`dbscan`** refleja que, con tres puntos para formar un cluster (`minPts`), ninguna de las opciones de $\epsilon$ evaluadas (`eps_grid`) genera clústeres excepto en la última: todos los casos presentan **0 clústeres** menos el último (`n_clusters`) por lo que tendran **18 puntos de ruido**excepto el último (`n_noise`), lo que implica que los datos no presentan densidades suficientemente diferenciadas para este método bajo dichos parámetros, las distancias se pueden observar en `kNN_dits`.

Las dos tablas finales permiten comparar visualmente estos resultados: la tabla de métodos basados en $k$ confirma la tendencia decreciente esperada de tracew en k-means y la variabilidad de las WSS en GMM y Spectral; mientras que la tabla específica para DBSCAN resume la incapacidad del método para encontrar grupos en este conjunto de datos.




In [ ]:
# Comparamos los resultados de los 4 métodos no jerárquicos
# Extraemos los resultados obtenidos
tabla_k = res_elbow$tablas$k_methods
tabla_dbscan = res_elbow$tablas$dbscan

In [ ]:
# Cargamos la librería 'knitr'
require(knitr)

In [ ]:
# Generamos la tabla comparativa de los métodos k-means, DBSCAN y GMM
kable(tabla_k, caption = "Resumen de índices para k-means, GMM y Spectral Clustering", digits = 3, align = "c")

**Interpretación.-** A partir de la tabla comparativa de los índices obtenidos por k-means, GMM y Spectral Clustering, se observar diversos  comportamiento de cada método.

En primer lugar, **k-means muestra un patrón claramente decreciente en tracew**, pasando de aproximadamente 50.7 en $k=2$ a 8.1 en $k=10$. La caída más pronunciada ocurre entre $k=3$ y $k=4$, lo que sugiere que el método identifica una estructura bien definida en torno a **4 grupos**. A partir de ese punto, las mejoras son más moderadas, coherentes con un "codo" en torno a $k=4$.

Por el contrario, tanto **GMM** como **Spectral Clustering** presentan **índices de dispersión más irregulares**. En GMM, los valores de WSS no exhiben un descenso progresivo; de hecho, algunos valores aumentan de forma notable (por ejemplo, en $k=4$ y $k=8$), lo que indica que las particiones generadas por los modelos de mezcla no producen agrupaciones más compactas conforme aumenta $k$. Spectral Clustering muestra un patrón similar, con fluctuaciones marcadas y sin evidencias de un punto de inflexión claro.

Comparando los tres métodos, se observa que **ningún método basado en WSS (GMM o Spectral) sugiere un número óptimo evidente de clústeres**, mientras que **k-means sí identifica una estructura relativamente estable**, apoyada en una reducción clara de la inercia hasta $k=4$. Esto sugiere que, para estos datos, k-means captura mejor la organización subyacente, mientras que los métodos basados en modelos probabilísticos o en la estructura espectral del grafo no logran detectar agrupaciones con la misma nitidez.

Aunque todavia que queda el método DBSCAN que se estudiará a continuación.


In [ ]:
# Generamos la tabla comparativa del método DBSCAN
kable(tabla_dbscan, caption = "Resumen de resultados para DBSCAN según eps", digits = 3, align = "c")

**Interpretación.-**  
El comportamiento de DBSCAN en este conjunto confirma que el parámetro $eps$ controla de forma decisiva la estructura que el algoritmo es capaz de detectar. A partir de los resultados observados pueden destacarse varios puntos:

* **Sensibilidad extrema a la densidad:** Para valores bajos y moderados de $eps$ (entre 0.40 y 1.15), el algoritmo no identifica ningún clúster y clasifica todos los puntos como ruido. Esto indica que, bajo criterios estrictos de densidad, los datos no presentan regiones suficientemente compactas como para formar grupos.

* **Umbral crítico alrededor de $eps \approx 1.20$:** Sólo cuando el radio de vecindad aumenta lo suficiente, DBSCAN detecta la presencia de **2 clústeres** y reduce significativamente la cantidad de puntos marcados como ruido (de 18 a 11). Esto refleja que la estructura del dataset no es densa, sino más bien dispersa, por lo que sólo agrupamientos “amplios” cumplen los requisitos de densidad.

* **Estabilidad del ruido:** El número de puntos considerados ruido permanece constante (18) para todos los valores hasta $eps=1.15$, lo cual sugiere que el patrón espacial carece de densidades locales destacables en ese rango.

En resumen, el análisis indica que **únicamente con un $eps$ suficientemente grande** aparece una estructura de **2 clústeres**, lo que sugiere que el dataset contiene grupos amplios pero poco densos. La elección de $eps$ debería de cambiara a valores superiores quizá una orquilla de 1.20 a 1.75 sería más obtima.


Ahora se mostrá en gráfica los valores para que se pueda observar de forma gráfica el "codo".

In [ ]:
# Cargamos las librerías necesarias para llevar a cabo la representación gráfica
library(ggplot2)
library(dplyr)
library(tidyr)

In [ ]:
# Extraemos la tabla de métodos basados en k
tabla_k = res_elbow$tablas$k_methods

In [ ]:
# Pasamos a formato largo para poder usar ggplot cómodamente
tabla_k_long = tabla_k %>%

# Cambiamos de formato "ancho" a "largo"
pivot_longer(
    cols = c(tracew_kmeans, wss_gmm, wss_spectral), # Indicamos qué columnas queremos “desapilar” (las medidas de WSS)
    names_to = "metodo",                            # El nombre de la nueva columna que contendrá el nombre del método (antes era el nombre de la variable)
    values_to = "wss"                               # El nombre de la nueva columna que contendrá los valores numéricos de esas medidas
  ) %>%

# Modificamos o creamos columnas en el data.frame resultante
mutate(
  metodo = recode(                                  # Reescribimos los valores de la columna 'metodo' para que sean más legibles
    metodo,
    "tracew_kmeans" = "k-means (tracew)",           # Cambiamos la etiqueta técnica 'tracew_kmeans' por una descripción más clara
    "wss_gmm"       = "GMM (WSS aprox.)",           # Cambiamos 'wss_gmm' por 'GMM (WSS aprox.)'
    "wss_spectral"  = "Spectral (WSS aprox.)"       # Cambiamos 'wss_spectral' por 'Spectral (WSS aprox.)'
  )
)

In [ ]:
# Representamos gráficamente el método del codo
ggplot(tabla_k_long, aes(x = k, y = wss, color = metodo)) +
  geom_line(linewidth = 0.8) +
  geom_point(size = 2) +
  labs(title = "Método del codo para k-means, GMM y Spectral Clustering",
       x     = "Número de clústeres (k)",
       y     = "WSS / tracew (variabilidad intra-clúster)",
       color = "Método") +
  theme_minimal(base_size = 12)

**Interpretación.-**  
El gráfico compara la evolución de la variabilidad intra–clúster (WSS o su aproximación) para **k-means**, **GMM** y **Spectral Clustering** conforme aumenta el número de clústeres $k$.  

En el caso de **k-means**, se observa una **disminución muy marcada entre $k=3$ y $k=4$**, seguida de mejoras cada vez más pequeñas para valores superiores de $k$. Este comportamiento es característico de un **punto de codo** claro en **$k=4$**, donde añadir más clústeres no reduce de forma significativa la variabilidad interna.

Por el contrario, las curvas de **GMM** y **Spectral Clustering** se mantienen **mucho más aleatoria**, con oscilaciones pero sin un descenso abrupto después de los primeros valores de $k$. Esto indica que estos métodos **no obtienen una ganancia sustancial** al incrementar el número de clústeres, por lo que el criterio del codo no resulta tan concluyente para ellos.

En conjunto, el comportamiento de las tres curvas sugiere que **$k=4$ es la elección más razonable**, respaldado por la señal clara dada por k-means y la ausencia de otra opción por **GMM** y **Spectral Clustering**.


In [ ]:
# Extraemos la tabla de los resultados de DBSCAN
tabla_db = res_elbow$tablas$dbscan

In [ ]:
# Evolución del número de clústeres según eps
ggplot(tabla_db, aes(x = eps, y = n_clusters)) +
  geom_line(linewidth = 0.8) +
  geom_point(size = 2) +
  labs(title = "DBSCAN: número de clústeres en función de eps",
       x     = "eps",
       y     = "Número de clústeres (sin ruido)") +
  scale_x_continuous(breaks = tabla_db$eps) +
  theme_minimal(base_size = 12)

**Interpretación.-**  
El gráfico muestra la evolución del número de clústeres detectados por **DBSCAN** en función del parámetro `eps`. Para un amplio rango inicial (`eps` entre 0.40 y 1.15), el algoritmo devuelve **cero clústeres válidos**, lo que significa que **todos los puntos son considerados ruido** bajo estos niveles de exigencia de densidad. Esto sugiere que, con `minPts = 3`, la distancia entre puntos es demasiado grande para formar núcleos densos en ese intervalo.

Solo cuando `eps` alcanza valores alrededor de **1.20** aparece finalmente una estructura con **dos clústeres**, lo que indica que el algoritmo empieza a considerar suficientes puntos dentro del radio para formar regiones densas coherentes.  

En conjunto, el gráfico evidencia que el conjunto de datos **no presenta densidades suficientemente compactas** como para que DBSCAN detecte clústeres en eps pequeños o moderados. La aparición de clústeres únicamente en eps grandes sugiere que la estructura es **poco densa** y requiere radios amplios para conectarse, por lo que la configuración de DBSCAN debe interpretarse con cautela en este caso.


In [ ]:
# Evolución del número de puntos ruido según eps
ggplot(tabla_db, aes(x = eps, y = n_noise)) +
  geom_line(linewidth = 0.8) +
  geom_point(size = 2) +
  labs(title = "DBSCAN: número de puntos ruido en función de eps",
       x     = "eps",
       y     = "Número de observaciones etiquetadas como ruido") +
  scale_x_continuous(breaks = tabla_db$eps) +
**Interpretación.-**
El gráfico muestra la evolución del número de clústeres detectados por **DBSCAN** en función del parámetro `eps`. Para un amplio rango inicial (`eps` entre 0.40 y 1.15), el algoritmo devuelve **cero clústeres válidos**, lo que significa que **todos los puntos son considerados ruido** bajo estos niveles de exigencia de densidad. Esto sugiere que, con `minPts = 3`, la distancia entre puntos es demasiado grande para formar núcleos densos en ese intervalo.

Solo cuando `eps` alcanza valores alrededor de **1.20** aparece finalmente una estructura con **dos clústeres**, lo que indica que el algoritmo empieza a considerar suficientes puntos dentro del radio para formar regiones densas coherentes.

En conjunto, el gráfico evidencia que el conjunto de datos **no presenta densidades suficientemente compactas** como para que DBSCAN detecte clústeres en eps pequeños o moderados. La aparición de clústeres únicamente en eps grandes sugiere que la estructura es **poco densa** y requiere radios amplios para conectarse, por lo que la configuración de DBSCAN debe interpretarse con cautela en este caso.
  theme_minimal(base_size = 12)


**Interpretación.-**  
La gráfica muestra la evolución del número de puntos etiquetados como **ruido** por DBSCAN en función del parámetro `eps`. Para todo el rango inicial (`eps` entre 0.40 y 1.15), el algoritmo considera **a las 18 observaciones como ruido**, lo cual indica que **no existe suficiente densidad local** para formar clústeres con `minPts = 3` en ese intervalo.

Solo cuando `eps` alcanza aproximadamente **1.20** el número de puntos ruido cae bruscamente hasta **11**, lo que significa que algunos puntos empiezan a encontrarse dentro de radios suficientemente amplios como para cumplir el criterio de densidad y formar un par de clústeres válidos.

En conjunto, el gráfico confirma que el conjunto de datos presenta una **densidad muy baja o muy dispersa**, de modo que DBSCAN no identifica regiones densas hasta valores de `eps` relativamente grandes. Esto refuerza la idea de que, para este dataset, DBSCAN solo ofrece soluciones interpretables cuando se permite un radio amplio que conecte los puntos.


Tras realizar el análisis del criterio del codo se realizará el criterio delestadistico GAP, como la función es propia se explicará que realiza la función.

La función NHC_gap sustituye el análisis anterior basado en el criterio del codo, por el Estadístico GAP, que es una medida más robusta y formal para determinar el número óptimo de clústeres. Mientras que el criterio del codo se basaba en observar visualmente la reducción de la variabilidad intra–clúster, esta función calcula el valor GAP mediante bootstrap, comparando la dispersión real de los datos con la dispersión esperada bajo una distribución de referencia uniforme. Esto permite seleccionar el número de clústeres no solo por el "codo", sino mediante un criterio estadístico con soporte teórico.

Además, la función generaliza el análisis a tres metodologías distintas: k-means, Gaussian Mixture Models (GMM) y Spectral Clustering, creando wrappers personalizados para que todos puedan evaluarse con clusGap. Para cada método, la función construye una tabla comparativa de valores GAP entre diferentes valores de $k$, lo que permite contrastar cómo cada algoritmo estructura los datos y si coinciden o divergen en la elección del número óptimo de clústeres. Esto supone una mejora respecto a la función anterior del codo, que solo evaluaba la variabilidad interna sin marco de comparación entre métodos.

Finalmente, la función mantiene un tratamiento especial para DBSCAN, ya que el Estadístico GAP no es aplicable a algoritmos basados en densidad. En lugar de forzar un criterio inadecuado, la función implementa un análisis sistemático del parámetro eps, registrando el número de clústeres y el número de puntos ruido para distintos valores. Con ello se obtiene una visión clara de cómo evoluciona la estructura de densidad, complementando el enfoque basado en $k$ de los métodos anteriores. El resultado final es un procedimiento unificado, más riguroso y comparativo, que supera ampliamente las limitaciones del análisis previo basado únicamente en el codo.




In [ ]:
#--------------------------------------------------------------
# Función general para estimar el Estadístico GAP en:
#  - k-means (usando clusGap)
#  - GMM (Mclust + wrapper para clusGap)
#  - Spectral Clustering (specc + wrapper para clusGap)
#  - DBSCAN (Se mantiene análisis de eps, ya que Gap no aplica a densidad sin k fijo)
#--------------------------------------------------------------
NHC_gap = function(data, k_min = 2, k_max = 10, B = 50, eps_grid = seq(0.4, 1.2, by = 0.05), minPts_dbscan = 3, seed = 123){

#----------------------------------------------------------
# 0) Comprobaciones y carga de librerías
#----------------------------------------------------------

# Aseguramos que 'data' sea una matriz numérica
data = as.matrix(data)

# Vector de valores de k a evaluar
k_values = k_min:k_max

# Establecemos semilla de reproducibilidad
set.seed(seed)

libs_necesarias = c("mclust", "kernlab", "dbscan", "cluster")

for (lib in libs_necesarias) {
  if (!requireNamespace(lib, quietly = TRUE)) {
    install.packages(lib)
  }
  library(lib, character.only = TRUE)
}

# Cargamos librerías necesarias
library(cluster)   # Para clusGap (k-means)
library(mclust)    # Para GMM
library(kernlab)   # Para Spectral Clustering
library(dbscan)    # Para DBSCAN

#----------------------------------------------------------
# 1) Definición de Wrappers (Funciones auxiliares para clusGap)
#    clusGap necesita funciones que acepten (x, k) y devuelvan
#    una lista con el componente $cluster
#----------------------------------------------------------

# Wrapper para GMM
gmm_fn = function(x, k){
  model = Mclust(x, G = k, verbose = FALSE)
  list(cluster = model$classification)
}

# Wrapper para Spectral Clustering
spec_fn = function(x, k){
  # Usamos tryCatch por si falla la convergencia en alguna iteración
  out = tryCatch({
    res = specc(x, centers = k, kernel = "rbfdot")
    list(cluster = res@.Data)
  }, error = function(e) {
    # En caso de error, devolvemos un cluster aleatorio (fallback)
    list(cluster = sample(1:k, nrow(x), replace = TRUE))
  })
  return(out)
}

# Wrapper para K-means (aseguramos nstart para estabilidad)
kmeans_fn = function(x, k){
  kmeans(x, centers = k, nstart = 25)
}

#----------------------------------------------------------
# 2) K-MEANS - Estadístico GAP
#----------------------------------------------------------
# Calculamos el Gap statistic con B iteraciones de bootstrap
gap_km_obj = clusGap(data, FUNcluster = kmeans_fn, K.max = k_max, B = B)

# Calculamos el Gap statistic con B iteraciones de bootstrap
# La tabla de clusGap empieza en k=1, filtramos por filas
gap_km_values = gap_km_obj$Tab[k_values, "gap"]

#----------------------------------------------------------
# 3) GMM - Estadístico GAP
#----------------------------------------------------------
# Nota: Esto puede tardar dependiendo del tamaño de datos y B
# Calculamos el Gap statistic con B iteraciones de bootstrap
gap_gmm_obj = clusGap(data, FUNcluster = gmm_fn, K.max = k_max, B = B)

# Calculamos el Gap statistic con B iteraciones de bootstrap
# La tabla de clusGap empieza en k=1, filtramos por filas
gap_gmm_values = gap_gmm_obj$Tab[k_values, "gap"]

#----------------------------------------------------------
# 4) SPECTRAL CLUSTERING - Estadístico GAP
#----------------------------------------------------------
# Nota: Esto puede tardar dependiendo del tamaño de datos y B
# Calculamos el Gap statistic con B iteraciones de bootstrap
gap_spec_obj = clusGap(data, FUNcluster = spec_fn, K.max = k_max, B = B)

# Calculamos el Gap statistic con B iteraciones de bootstrap
# La tabla de clusGap empieza en k=1, filtramos por filas
gap_spec_values = gap_spec_obj$Tab[k_values, "gap"]

#----------------------------------------------------------
# 5) DBSCAN - Análisis de sensibilidad (eps)
#    El estadístico Gap no es aplicable directamente porque
#    DBSCAN no fija k. Mantenemos el análisis de eps/ruido.
#----------------------------------------------------------
# Distancias al minPts-ésimo vecino (para visualización)
kNN_dists = kNNdist(data, k = minPts_dbscan)
kNN_dists = sort(kNN_dists)

# Evaluamos la rejilla de eps
n_clusters_db = numeric(length(eps_grid))
n_noise_db    = numeric(length(eps_grid))

# Ajustamos DBSCAN para cada eps de la rejilla
for (i in seq_along(eps_grid)) {
  eps_i = eps_grid[i]
  db_i = dbscan(data, eps = eps_i, minPts = minPts_dbscan)

  # Clusters excluyendo ruido (0)
  n_clusters_db[i] = length(setdiff(unique(db_i$cluster), 0))
  n_noise_db[i]    = sum(db_i$cluster == 0)
}

#----------------------------------------------------------
# 6) Construimos tablas resumen comparativas
#----------------------------------------------------------
# Tabla comparativa de valores GAP
tabla_gap = data.frame(k = k_values, gap_kmeans = gap_km_values, gap_gmm = gap_gmm_values, gap_spectral = gap_spec_values)

# Tabla comparativa para DBSCAN
tabla_dbscan = data.frame(eps = eps_grid, n_clusters = n_clusters_db, n_noise = n_noise_db)

#----------------------------------------------------------
# 7) Devolvemos resultados en lista
#----------------------------------------------------------
resultados = list(
  gap_objects = list(
    kmeans   = gap_km_obj,    # Objeto completo clusGap (contiene SE.sim, etc.)
    gmm      = gap_gmm_obj,
    spectral = gap_spec_obj
  ),
  dbscan = list(
    minPts     = minPts_dbscan,
    kNN_dists  = kNN_dists,
    eps_grid   = eps_grid,
    n_clusters = n_clusters_db,
    n_noise    = n_noise_db
  ),
  tablas = list(
    gap_methods = tabla_gap,   # Tabla resumen de valores Gap
    dbscan      = tabla_dbscan # Tabla resumen DBSCAN
  )
)

# Devuelve el resultado final
return(resultados)
}


In [ ]:
# Aplicamos la función NHC_gap sobre nuestra base de datos estandarizada (puede tardar unos minutos)
res_gap = NHC_gap(datos_est)

In [ ]:
# Información obtenida con la función 'NHC_gap'
str(res_gap)

In [ ]:
# Información obtenida con la función 'NHC_gap' (vista completa)
show(res_gap)

**Interpretación.-** El objeto generado por la función `NHC_gap` contiene **tres bloques de resultados**: los estadísticos Gap para los métodos basados en $k$ (*k-means*, *GMM* y *Spectral Clustering*), el análisis de sensibilidad para **DBSCAN**, y dos tablas resumen listas para análisis comparativo. En conjunto, este resultado permite evaluar la estructura de clústeres desde una perspectiva estadística más sólida que el criterio del codo, ya que compara la dispersión observada con la obtenida bajo una distribución de referencia. Con de $50$ iteraciones bootstrap  (`B`).

Dentro del apartado **`gap_objects`**, observamos que los tres métodos (*k-means*, *GMM* y *Spectral*) seleccionan **$k = 1$** como número óptimo de clústeres bajo el criterio *firstSEmax*, ya que el valor del estadístico Gap es máximo en $k = 1$ y decrece conforme aumenta $k$. Esto es coherente con las tablas: en los tres métodos, el estadístico Gap es positivo únicamente en $k = 1$, y en la mayoría de los casos se vuelve negativo a partir de $k = 2$, indicando que añadir más clústeres no mejora la separación respecto al modelo nulo. Además, el patrón decreciente o fluctuante de la columna `gap` en `Tab` confirma que no hay evidencia estadística de múltiples grupos bien definidos en los datos según ninguno de los tres algoritmos.

En cuanto al bloque **`dbscan`**, el análisis de densidad muestra una situación similar. Para la rejilla de valores de $\epsilon$ definida (`eps_grid`), el método produce **0 clústeres** en todos los casos salvo el último, donde encuentra únicamente **2 clústeres** (`n_clusters`) y reduce el número de puntos de ruido a 11 (`n_noise`). Esto implica que, para la mayor parte del rango explorado, las distancias entre puntos (`kNN_dists`) no son suficientemente pequeñas como para formar regiones densas estables, la cual se definio como 3 equipos para un cluster (`minPts`) y el método no detecta estructura agregada relevante bajo los parámetros elegidos.

Finalmente, las dos tablas incluidas en **`tablas`** sintetizan estas conclusiones. La tabla **`gap_methods`** muestra que para $k = 2,\dots,10$ todos los valores del estadístico Gap son pequeños y mayoritariamente negativos, reforzando que ninguno de los métodos identifica agrupamientos significativos más allá de $k = 1$. Por su parte, la tabla **`dbscan`** confirma la ausencia de clústeres en la mayoría de configuraciones de $\epsilon$. En conjunto, este objeto indica que, según el criterio Gap y el análisis de densidad, **los datos no presentan una estructura clara de clústeres**, o bien esta es demasiado débil para ser detectada por los métodos considerados.






In [ ]:
# Comparamos los resultados de los 4 métodos no jerárquicos
# Extraemos la tabla del Estadístico Gap
tabla_gap = res_gap$tablas$gap_methods

In [ ]:
# Generamos la tabla comparativa de los métodos k-means, DBSCAN y GMM (Estadístico Gap)
kable(tabla_gap, caption = "Resumen del Estadístico Gap para k-means, GMM y Spectral Clustering", digits = 4, align = "c",
      col.names = c("k", "k-means (Gap)", "GMM (Gap)", "Spectral (Gap)"))

**Interpretación.-** La tabla muestra los valores del **Estadístico Gap** para distintos valores de $k$ usando *k-means*, *GMM* y *Spectral Clustering*. En este caso, los valores del Gap son muy pequeños y **no presentan máximos positivos claros**, lo que indica que los datos **no muestran una estructura de clústeres fuerte** según esta métrica.

Ninguno de los tres métodos exhibe un incremento sustancial del Gap que sugiera la presencia de agrupamientos bien definidos. De hecho:

- En **k-means**, los valores del Gap son ligeramente positivos solo en $k=2$, y desde $k=3$ en adelante son todos negativos, indicando que añadir más clústeres **no mejora la separación respecto al modelo nulo** ya que un valor negativo significa que es peor que una elección aleatoria de clusters.
- En **GMM**, ocurre algo similar: los valores oscilan en torno a cero y, aunque algunos son positivos (por ejemplo, en $k=4$ y $k=5$), no siguen un patrón creciente ni definen un máximo destacable.
- En **Spectral Clustering**, los valores positivos aparecen en algunos valores de $k$ (como $k=3$, $k=4$ y $k=7$), pero no forman una tendencia consistente ni estable a través de los valores de $k$.

Dado que el **criterio del Gap busca un máximo bien definido**, la ausencia de un pico claro implica que el conjunto de datos **no presenta una estructura de clústeres claramente identificable** bajo estos métodos. En consecuencia, **no hay evidencia sólida para seleccionar un valor óptimo de $k$**, y cualquier elección debería basarse en otros criterios (codo y silueta promedio).

En resumen, el Estadístico Gap sugiere que **los datos no contienen una estructura de clústeres fuerte o bien separada**, y que **ningún valor de $k$ destaca como óptimo** según este criterio.


In [ ]:
# Generamos la tabla comparativa del método DBSCAN (Estadístico Gap)
kable(res_gap$tablas$dbscan, caption = "Resumen de resultados para DBSCAN según eps (Estadístico Gap)", digits = 3, align = "c")

**Interpretación.-** Los resultados muestran que **DBSCAN no identifica ningún clúster** en la mayor parte del rango de valores de `eps`. Para todos los radios entre 0.40 y 1.15, el algoritmo considera que **todos los puntos son ruido**, lo que implica que la densidad local nunca es lo suficientemente alta como para formar grupos según el criterio interno del algoritmo. Este comportamiento indica que, en este conjunto de datos, **no existen zonas de alta densidad separadas entre sí** bajo los valores de vecindad evaluados.

Solo cuando `eps = 1.20` se detecta una estructura no trivial: aparecen **2 clústeres** y el número de puntos clasificados como ruido se reduce a 11. Esto sugiere que solo con un radio de vecindad relativamente grande, es decir, permitiendo que muchos puntos entren dentro del mismo entorno de densidad, DBSCAN es capaz de encontrar grupos cohesivos. Sin embargo, este valor elevado de `eps` también implica que la separación entre los clústeres es débil y que el algoritmo está forzando la unión de regiones que solo parecen densas a gran escala.

En conjunto, el análisis indica que **DBSCAN no detecta una estructura de densidad clara en los datos** salvo cuando se incrementa el radio de manera muy amplia. Por tanto, este algoritmo **no aporta evidencia sólida de clústeres bien definidos**, y confirma que los métodos basados en distancia o conectividad (como k-means, GMM o spectral) son más adecuados para este conjunto de datos.


Al igual que en el criterio anterior se procederá a graficarlos.

In [ ]:
# Extraemos la tabla de resultados GAP
tabla_gap = res_gap$tablas$gap_methods

In [ ]:
# Pasamos a formato largo para ggplot
tabla_gap_long = tabla_gap %>%
  pivot_longer(
    cols      = c(gap_kmeans, gap_gmm, gap_spectral),
    names_to  = "metodo",
    values_to = "gap_stat"
  ) %>%
  mutate(
    metodo = recode(
      metodo,
      "gap_kmeans"   = "k-means (Gap)",
      "gap_gmm"      = "GMM (Gap)",
      "gap_spectral" = "Spectral (Gap)"
    )
  )

In [ ]:
# Representamos gráficamente el Estadístico Gap
ggplot(tabla_gap_long, aes(x = k, y = gap_stat, color = metodo)) +
  geom_line(linewidth = 0.8) +
  geom_point(size = 2) +
  labs(title = "Estadístico Gap para k-means, GMM y Spectral Clustering",
       subtitle = "Valores más altos indican una mejor estructura de clústeres",
       x     = "Número de clústeres (k)",
       y     = "Estadístico Gap",
       color = "Método") +
  scale_x_continuous(breaks = tabla_gap$k) +
  theme_minimal(base_size = 12)

**Interpretación.-** La representación del **estadístico Gap** evidencia un patrón consistente entre los tres métodos evaluados, destacando de forma clara que la estructura más sólida se alcanza en $k=4$. El método **Spectral Clustering** (línea azul) presenta su **máximo global** en este punto, lo que indica que, según la conectividad y las relaciones no lineales entre observaciones, la partición en tres grupos es la más marcada. Por su parte, **GMM** muestra también un **pico local pronunciado** en $k=3$, antes de que el valor del Gap comience a disminuir progresivamente para valores superiores de $k$.

En el caso de **k-means**, exceptuando en $k=2$ los valores son negativos indicando que es mejor la selección de clusters de forma aleatoria.



In [ ]:
# Extraemos la tabla de DBSCAN del objeto res_gap
tabla_gap_db = res_gap$tablas$dbscan

In [ ]:
# Nota: Como DBSCAN no usa Gap directo, visualizamos la estabilidad estructural
# Evolución del número de clústeres según eps
ggplot(tabla_gap_db, aes(x = eps, y = n_clusters)) +
  geom_line(linewidth = 0.8, color = "darkblue") +
  geom_point(size = 2, color = "darkblue") +
  labs(title = "DBSCAN: Estabilidad de clústeres según eps",
       x     = "eps (Radio de vecindad)",
       y     = "Número de clústeres formados") +
  scale_x_continuous(breaks = tabla_gap_db$eps) +
  theme_minimal(base_size = 12)

**Interpretación.-**  
El gráfico muestra la evolución del número de clústeres detectados por **DBSCAN** en función del parámetro `eps`. Para un amplio rango inicial (`eps` entre 0.40 y 1.15), el algoritmo devuelve **cero clústeres válidos**, lo que significa que **todos los puntos son considerados ruido** bajo estos niveles de exigencia de densidad. Esto sugiere que, con `minPts = 3`, la distancia entre puntos es demasiado grande para formar núcleos densos en ese intervalo.

Solo cuando `eps` alcanza valores alrededor de **1.20** aparece finalmente una estructura con **dos clústeres**, lo que indica que el algoritmo empieza a considerar suficientes puntos dentro del radio para formar regiones densas coherentes.  

En conjunto, el gráfico evidencia que el conjunto de datos **no presenta densidades suficientemente compactas** como para que DBSCAN detecte clústeres en eps pequeños o moderados. La aparición de clústeres únicamente en eps grandes sugiere que la estructura es **poco densa** y requiere radios amplios para conectarse, por lo que la configuración de DBSCAN debe interpretarse con cautela en este caso.





In [ ]:
# Evolución del ruido según eps
ggplot(tabla_gap_db, aes(x = eps, y = n_noise)) +
  geom_line(linewidth = 0.8, color = "firebrick") +
  geom_point(size = 2, color = "firebrick") +
  labs(title = "DBSCAN: Cantidad de ruido según eps",
       x     = "eps (Radio de vecindad)",
       y     = "Puntos considerados ruido") +
  scale_x_continuous(breaks = tabla_gap_db$eps) +
  theme_minimal(base_size = 12)

**Interpretación.-**  
La gráfica muestra la evolución del número de puntos etiquetados como **ruido** por DBSCAN en función del parámetro `eps`. Para todo el rango inicial (`eps` entre 0.40 y 1.15), el algoritmo considera **a las 18 observaciones como ruido**, lo cual indica que **no existe suficiente densidad local** para formar clústeres con `minPts = 3` en ese intervalo.

Solo cuando `eps` alcanza aproximadamente **1.20** el número de puntos ruido cae bruscamente hasta **11**, lo que significa que algunos puntos empiezan a encontrarse dentro de radios suficientemente amplios como para cumplir el criterio de densidad y formar un par de clústeres válidos.

En conjunto, el gráfico confirma que el conjunto de datos presenta una **densidad muy baja o muy dispersa**, de modo que DBSCAN no identifica regiones densas hasta valores de `eps` relativamente grandes. Esto refuerza la idea de que, para este dataset, DBSCAN solo ofrece soluciones interpretables cuando se permite un radio amplio que conecte los puntos.

Ya que la función es propia se dará una explicación de lo que hace. La función NHC_elbow constituye una herramienta integrada para aplicar el método del codo en varios algoritmos de clustering no jerárquico: k-means, Gaussian Mixture Models (GMM), Spectral Clustering y DBSCAN. Su estructura está diseñada para garantizar reproducibilidad, control del rango de búsqueda y comparabilidad entre métodos. A partir del conjunto de datos introducido, la función calcula múltiples particiones para un rango de valores de $k$ y para los distintos algoritmos: **k-means**, **GMM** y **Spectral Clustering**. Para cada combinación, obtiene métricas internas de validación como el **índice de silueta**, el **índice de Dunn**, el **Davies–Bouldin**, el **Calinski–Harabasz**, el **Gap Statistic** y otras medidas relevantes que permiten comparar la calidad de la estructura encontrada. Además, organiza todos los resultados en tablas y listas para incorporar en informes de forma inmediata sin neceisidad de generación externa, que sintetizan el comportamiento de los métodos frente al número de clústeres.



In [ ]:
#--------------------------------------------------------------
# Función general para calcular la Silueta Promedio en:
#  - k-means (stats + cluster)
#  - GMM (mclust + cluster)
#  - Spectral Clustering (kernlab + cluster)
#  - DBSCAN (dbscan + cluster)
#--------------------------------------------------------------
NHC_silueta = function(data, k_min = 2, k_max = 10, eps_grid = seq(0.4, 1.2, by = 0.05), minPts_dbscan = 3, seed = 123){

#----------------------------------------------------------
# 0) Comprobaciones y carga de librerías
#----------------------------------------------------------

# Aseguramos que 'data' sea una matriz numérica
data = as.matrix(data)

# Calculamos la matriz de distancias una sola vez (necesaria para silhouette)
# Usamos distancia Euclídea por defecto
dist_mat = dist(data)

# Vector de valores de k a evaluar
k_values = k_min:k_max

# Establecemos semilla de reproducibilidad
set.seed(seed)

libs_necesarias = c("mclust", "kernlab", "dbscan", "cluster")

for (lib in libs_necesarias) {
  if (!requireNamespace(lib, quietly = TRUE)) {
    install.packages(lib)
  }
  library(lib, character.only = TRUE)
}

# Cargamos librerías necesarias
library(cluster)   # Para la función silhouette()
library(mclust)    # Para GMM
library(kernlab)   # Para Spectral Clustering
library(dbscan)    # Para DBSCAN

#----------------------------------------------------------
# 1) Función auxiliar: extrae silueta promedio
#----------------------------------------------------------
calc_sil_avg = function(cl, d_mat) {

# La silueta solo se define para k >= 2
# Si hay 1 solo grupo o todo es ruido, devuelve NA o 0
k_enc = length(unique(cl))
  if (k_enc < 2) {
    return(NA)
  } else {
    sil_obj = silhouette(cl, d_mat)
    # Devolvemos la media de la tercera columna (width)
    return(summary(sil_obj)$avg.width)
  }
}

#----------------------------------------------------------
# 2) K-MEANS - Silueta Promedio
#----------------------------------------------------------
sil_km = numeric(length(k_values))

for (i in seq_along(k_values)) {
  k = k_values[i]

  # Ejecutamos k-means
  km_res = kmeans(data, centers = k, nstart = 25)

  # Calculamos silueta
  sil_km[i] = calc_sil_avg(km_res$cluster, dist_mat)
}

#----------------------------------------------------------
# 3) GMM (Mclust) - Silueta Promedio
#----------------------------------------------------------
sil_gmm = numeric(length(k_values))

for (i in seq_along(k_values)) {
  k = k_values[i]

  # Ajustamos GMM
  gmm_res = Mclust(data, G = k, verbose = FALSE)

  # Clasificación
  cl = gmm_res$classification

  # Nota: Calculamos la silueta sobre la distancia Euclídea original
  # para hacer los métodos comparables en términos de compacidad geométrica.
  sil_gmm[i] = calc_sil_avg(cl, dist_mat)
}

#----------------------------------------------------------
# 4) SPECTRAL CLUSTERING - Silueta Promedio
#----------------------------------------------------------
sil_spectral = numeric(length(k_values))

for (i in seq_along(k_values)) {
  k = k_values[i]

  # Ajustamos Spectral
  # Usamos tryCatch para evitar paradas si no converge
  spec_res = tryCatch({
    specc(data, centers = k, kernel = "rbfdot")
  }, error = function(e) NULL)

  if (!is.null(spec_res)) {
    cl = spec_res@.Data
    sil_spectral[i] = calc_sil_avg(cl, dist_mat)
  } else {
    sil_spectral[i] = NA
  }
}

#----------------------------------------------------------
# 5) DBSCAN - Silueta Promedio por eps
#----------------------------------------------------------
# Para DBSCAN, calcularemos la silueta de los puntos CLASIFICADOS.
# Opción: Excluir el ruido (cluster 0) del cálculo de la media,
# ya que el ruido no forma un clúster cohesivo.

sil_dbscan    = numeric(length(eps_grid))
n_clusters_db = numeric(length(eps_grid))
n_noise_db    = numeric(length(eps_grid))

# Distancias kNN para referencia visual del codo (igual que en funciones previas)
kNN_dists = kNNdist(data, k = minPts_dbscan)
kNN_dists = sort(kNN_dists)

for (i in seq_along(eps_grid)) {
  eps_i = eps_grid[i]

  db_res = dbscan(data, eps = eps_i, minPts = minPts_dbscan)
  cl = db_res$cluster

  # Guardamos métricas estructurales
  n_clusters_db[i] = length(setdiff(unique(cl), 0))
  n_noise_db[i]    = sum(cl == 0)

  # Calculamos silueta SOLO si hay al menos 2 clústeres reales (excluyendo ruido)
  # O si consideramos ruido como grupo, dependerá de la interpretación.
  # Aquí: filtramos el ruido para ver la calidad de los grupos formados.
  mask_no_ruido = cl != 0

  if (length(unique(cl[mask_no_ruido])) >= 2) {
    # Calculamos silueta solo con los datos que no son ruido
    # Necesitamos subconjunto de la matriz de distancias también
    dist_subset = as.dist(as.matrix(dist_mat)[mask_no_ruido, mask_no_ruido])
    sil_val = calc_sil_avg(cl[mask_no_ruido], dist_subset)
    sil_dbscan[i] = sil_val
  } else {
    sil_dbscan[i] = NA # No aplica silueta si hay < 2 grupos
  }
}

#----------------------------------------------------------
# 6) Construimos tablas resumen comparativas
#----------------------------------------------------------

# Tabla comparativa para métodos basados en k
tabla_sil = data.frame(k = k_values, sil_kmeans = sil_km, sil_gmm = sil_gmm, sil_spectral = sil_spectral)

# Tabla comparativa para DBSCAN
tabla_dbscan = data.frame(eps = eps_grid, n_clusters = n_clusters_db, n_noise = n_noise_db, avg_sil_clu = sil_dbscan)

#----------------------------------------------------------
# 7) Devolvemos resultados en lista estructurada
#----------------------------------------------------------
resultados = list(
  metrica = "Average Silhouette Width",
  dbscan_info = list(
    minPts    = minPts_dbscan,
    kNN_dists = kNN_dists,
    nota      = "La silueta de DBSCAN se calcula excluyendo puntos de ruido (cluster 0)"
  ),
  tablas = list(
    sil_methods = tabla_sil,    # Tabla resumen K-means, GMM, Spectral
    dbscan      = tabla_dbscan  # Tabla resumen DBSCAN
  )
)

# Devuelve el resultado final
return(resultados)
}


In [ ]:
# Aplicamos la función NHC_silueta sobre nuestra base de datos estandarizada
res_silueta = NHC_silueta(datos_est)

In [ ]:
# Información obtenida con la función 'NHC_silueta'
str(res_silueta)

In [ ]:
# Información obtenida con la función 'NHC_silueta' (vista completa)
show(res_silueta)

**Interpretación.-** El objeto resultante `res_silueta` contiene 3 elementos principales que resumen la evaluación basada en el Average Silhouette Width para los métodos de clustering considerados, incluyendo una sección específica dedicada a DBSCAN. El primer elemento, **`metrica`**, indica claramente que la medida empleada para comparar la calidad de los agrupamientos es **"Average Silhouette Width"** (Anchura de Silueta Promedio), una métrica que evalúa simultáneamente la coherencia interna de los clústeres y la separación entre ellos.

En el apartado **`dbscan_info`**, se especifica el valor de `minPts = 3`, correspondiente al número mínimo de vecinos requeridos para formar un clúster en DBSCAN. A continuación, se muestran las distancias a los 3 vecinos más cercanos (`kNN_dists`), donde los valores oscilan aproximadamente entre $1.49$ y $2.63$, lo que permite inferir el rango razonable de $\epsilon$ para evaluar densidades consistentes. La nota incluida (`nota`) aclara que la silueta de DBSCAN se computa excluyendo los puntos clasificados como ruido, es decir, aquellos asignados al clúster $0$.

Dentro del apartado **`tablas`**, el subelemento sil_methods presenta las anchuras medias de silueta para **k-means, GMM y Spectral Clustering** para $k = 2,\dots,10$. Los valores muestran que los tres métodos alcanzan sus puntuaciones más altas alrededor de $k = 4–6$, especialmente k-means con un máximo en $k = 6$ (`sil_kmeans = 0.2708`), seguido muy de cerca por GMM y, en menor medida, por Spectral Clustering. Este comportamiento sugiere que, desde la métrica de silueta, los datos presentan una estructura moderadamente definida en ese rango de particiones, con cierta pérdida de coherencia para valores mayores de $k$.

Finalmente, el subelemento `dbscan` resume la evolución de DBSCAN para distintos valores de $\epsilon$. Se observa que todas las configuraciones entre $0.40$ y $1.15$ producen **0 clústeres y 18 puntos de ruido**, lo que implica que con esos radios no se alcanza suficiente densidad para formar agrupamientos estables (`n_clusters`, `n_noise`). Sólo en $\epsilon = 1.20$ el algoritmo identifica **2 clústeres** con **11 puntos de ruido**, alcanzando además una **silueta media elevada** (`avg_sil_clu = 0.44097`), lo que indica que, aunque DBSCAN es muy sensible al parámetro $\epsilon$, existe al menos una configuración que revela una estructura de densidad coherente en los datos.
  
  

In [ ]:
# Comparamos los resultados de los 4 métodos no jerárquicos
# Extraemos la tabla de la Silueta Promedio
tabla_k_silueta = res_silueta$tablas$sil_methods

In [ ]:
# Generamos la tabla comparativa de los métodos k-means, DBSCAN y GMM (Silueta promedio)
kable(tabla_k_silueta, caption = "Resumen de índices para k-means, GMM y Spectral Clustering (Silueta promedio)", digits = 3, align = "c")

Antes de realizar la interpretación cabe resaltar que la interpretación del valor será la <u>regla empírica comúnmente aceptada</u> la cual es:

- Si la silueta promedio está entre $0.71$ y $1$ entonces la estructura de clasificación encontrada es fuerte.

- Si la silueta promedio está entre $0.51$ y $0.7$ entonces la estructura de clasificación encontrada es razonable (aceptable).

- Si la silueta promedio está entre $0.26$ y $0.50$ entonces la estructura de clasificación encontrada es débil y puede ser artificial.

- Si la silueta promedio es $< 0.25$ entonces no se ha encontrado una estructura de clasificación.

**Interpretación.-** La tabla recoge las siluetas promedio de **k-means**, **GMM** y **Spectral Clustering** para $k = 2,\dots,10$, permitiendo evaluar la calidad relativa de cada partición. En primer lugar, se observa que los valores máximos se concentran en $k=6$, lo que indica que la estructura de los datos es moderada. El mayor valor de toda la tabla corresponde a **K-means** (`sil_kmeans = 0.271`), aunque este valor se encuentra por minimamente del umbral mínimo de $0.25$, por lo que no constituye evidencia suficiente de una estructura fuerte.

En cuanto a **Spectral Clustering** y **GMM**, ambos métodos muestran un comportamiento muy similar: las siluetas aumentan gradualmente hasta alcanzar valores moderados en **$k=4$** (`0.239` en Spectral Clustering y `0.246` en GMM) y alcanzan su máximo en **$k=6$** (`0.247` y `0.255`, respectivamente). Sin embargo, incluso estos máximos permanecen dentro del rango de **estructura no encontrada**, lo que indica que la partición solo está respaldada de manera tenue por la forma geométrica de los datos. A partir de ahí, todos los métodos muestran una degradación progresiva.

En conjunto, aunque ninguno de los métodos obtiene valores de Silueta que indiquen **estructura fuerte** ni **razonable**, se aprecia que los valores más consistentes y altos se concentran entre **$k = 4$ y $k = 6$**, siendo **k-means y GMM con $k=6$** los que alcanzan los mejores resultados relativos. Por tanto, estos análisis apuntan a que la estructura de los datos es **débil** pero más estable en torno a ese rango.


In [ ]:
# Generamos la tabla comparativa del método DBSCAN (Silueta promedio)
kable(res_silueta$tablas$dbscan, caption = "Resumen de resultados para DBSCAN según eps (Silueta promedio)", digits = 3, align = "c")

**Interpretación.-** La tabla resume cómo evoluciona el comportamiento de **DBSCAN** al variar el parámetro $\epsilon$ manteniendo `minPts = 3`. Se observa que para todos los valores comprendidos entre `eps = 0.40` y `eps = 1.15`, el algoritmo detecta **0 clústeres** (`n_clusters = 0`) y clasifica a **las 18 observaciones como ruido** (`n_noise = 18`). En consecuencia, la silueta promedio no puede calcularse (`avg_sil_clu = NA`), ya que DBSCAN no llega a formar ningún grupo válido. Este comportamiento indica que, en ese rango de vecindad, los puntos nunca alcanzan una densidad suficiente como para generar clústeres estables.

La única excepción se produce en `eps = 1.20`, donde el algoritmo identifica **2 clústeres** (`n_clusters = 2`) y reduce el número de puntos ruido a **11** (`n_noise = 11`). En este caso sí es posible calcular la silueta, obteniéndose un valor de **0.441** (`avg_sil_clu = 0.4409738`), lo que según las reglas empíricas corresponde a una **estructura de clústeres débil pero existente**. Este resultado pone de manifiesto que solo con valores de $\epsilon$ considerablemente grandes DBSCAN consigue detectar alguna estructura en los datos, aunque dicha estructura no alcanza niveles altos de separación o compacidad.


Se procederá a graficar los resultados.

In [ ]:
# Extraemos la tabla de resultados de Silueta
library(dplyr)
library(tidyr)
tabla_sil = res_silueta$tablas$sil_methods

In [ ]:

# Pasamos a formato largo
tabla_sil_long = tabla_sil %>%
  pivot_longer(
    cols      = c(sil_kmeans, sil_gmm, sil_spectral),
    names_to  = "metodo",
    values_to = "silueta"
  ) %>%
  mutate(
    metodo = recode(
      metodo,
      "sil_kmeans"   = "k-means (Silueta)",
      "sil_gmm"      = "GMM (Silueta)",
      "sil_spectral" = "Spectral (Silueta)"
    )
  )

In [ ]:
# Representamos gráficamente la Silueta Promedio
ggplot(tabla_sil_long, aes(x = k, y = silueta, color = metodo)) +
  geom_line(linewidth = 0.8) +
  geom_point(size = 2) +
  labs(title = "Silueta Promedio para k-means, GMM y Spectral Clustering",
       subtitle = "Valores cercanos a 1 indican clústeres densos y bien separados",
       x     = "Número de clústeres (k)",
       y     = "Anchura de Silueta Promedio",
       color = "Método") +
  scale_x_continuous(breaks = tabla_sil$k) +
  theme_minimal(base_size = 12)

**Interpretación.-** El análisis de la **silueta promedio** indica que el número óptimo de clústeres se alcanza en **k = 6**, donde los tres algoritmos logran sus valores máximos de separación entre grupos y cohesión interna. En este punto, **k-means** obtiene la mayor calidad de partición con una silueta de **0.271**, seguido de **GMM** con **0.255**, y **Spectral Clustering** con **0.247**, lo que sugiere que, aunque las diferencias no son grandes, k-means produce los clústeres más compactos y mejor definidos para este valor de k.

A partir de $k > 6$, las curvas de los tres métodos muestran un descenso progresivo en la calidad de las soluciones, indicando que añadir más clústeres produce agrupamientos menos cohesionados y peor separados. Por el contrario, para valores menores de k, la estructura de los datos aún no está completamente capturada y la silueta es inferior en todos los casos. La convergencia de los máximos en **k = 6** refuerza que este es el punto donde la partición ofrece el equilibrio más robusto entre complejidad del modelo y calidad del agrupamiento.


In [ ]:
# Extraemos la tabla DBSCAN del objeto res_silueta
tabla_sil_db = res_silueta$tablas$dbscan

In [ ]:
# Gráfico específico para Silueta en DBSCAN
# Nota: Los NAs (donde no hubo suficientes clusters) no se pintarán, cortando la línea,
# lo cual es correcto para indicar que ahí no hay solución válida.
ggplot(tabla_sil_db, aes(x = eps, y = avg_sil_clu)) +
  geom_line(linewidth = 0.8, color = "purple") +
  geom_point(size = 2, color = "purple") +
  labs(title = "DBSCAN: Calidad de los clústeres (Silueta) según eps",
       subtitle = "Calculado excluyendo el ruido. Espacios vacíos indican < 2 clústeres.",
       x     = "eps (Radio de vecindad)",
       y     = "Silueta Promedio (sin ruido)") +
  scale_x_continuous(breaks = tabla_sil_db$eps) +
  theme_minimal(base_size = 12)

**Interpretación.-**  
El gráfico muestra la evolución del número de clústeres detectados por **DBSCAN** en función del parámetro `eps`. Para un amplio rango inicial (`eps` entre 0.40 y 1.15), el algoritmo devuelve **cero clústeres válidos**, lo que significa que **todos los puntos son considerados ruido** bajo estos niveles de exigencia de densidad. Esto sugiere que, con `minPts = 3`, la distancia entre puntos es demasiado grande para formar núcleos densos en ese intervalo.

Solo cuando `eps` alcanza valores alrededor de **1.20** aparece finalmente una estructura con **dos clústeres**, lo que indica que el algoritmo empieza a considerar suficientes puntos dentro del radio para formar regiones densas coherentes.  

En conjunto, el gráfico evidencia que el conjunto de datos **no presenta densidades suficientemente compactas** como para que DBSCAN detecte clústeres en eps pequeños o moderados. La aparición de clústeres únicamente en eps grandes sugiere que la estructura es **poco densa** y requiere radios amplios para conectarse, por lo que la configuración de DBSCAN debe interpretarse con cautela en este caso.





In [ ]:
# Evolución del ruido según eps
ggplot(tabla_sil_db, aes(x = eps, y = n_noise)) +
  geom_line(linewidth = 0.8, color = "purple") +
  geom_point(size = 2, color = "purple") +
  labs(title = "DBSCAN: Cantidad de ruido según eps",
       x     = "eps (Radio de vecindad)",
       y     = "Puntos considerados ruido") +
  scale_x_continuous(breaks = tabla_sil_db$eps) +
  theme_minimal(base_size = 12)

**Interpretación.-**  
La gráfica muestra la evolución del número de puntos etiquetados como **ruido** por DBSCAN en función del parámetro `eps`. Para todo el rango inicial (`eps` entre 0.40 y 1.15), el algoritmo considera **a las 18 observaciones como ruido**, lo cual indica que **no existe suficiente densidad local** para formar clústeres con `minPts = 3` en ese intervalo.

Solo cuando `eps` alcanza aproximadamente **1.20** el número de puntos ruido cae bruscamente hasta **11**, lo que significa que algunos puntos empiezan a encontrarse dentro de radios suficientemente amplios como para cumplir el criterio de densidad y formar un par de clústeres válidos.

En conjunto, el gráfico confirma que el conjunto de datos presenta una **densidad muy baja o muy dispersa**, de modo que DBSCAN no identifica regiones densas hasta valores de `eps` relativamente grandes. Esto refuerza la idea de que, para este dataset, DBSCAN solo ofrece soluciones interpretables cuando se permite un radio amplio que conecte los puntos.

En este apartado se añadirán las siluetas individuales por medio de un **gráfico de siluetas**, dicho gráfico  muestra qué tan bien se **ajusta cada observación** al **clúster** que ha sido **asignado comparando qué tan cerca** se **encuentra** de las **demás en su cluster**.Se realizará para los $k=6$ y $eps=1.20$ ya que han sido los valores óptimos.

De cada uno se realizarán dos representaciones aunque ambas son equivalentes se realizan para ayudar en la comprensión.

In [ ]:
# Cargamos las librerías necesarias
library(cluster) # Contiene la función silhouette()
install.packages("factoextra")
library(factoextra) # Útil para la función fviz_silhouette (más estética)

In [ ]:
# Asignaciones de clústeres
# k=6 es el valor que utilizaremos para k-means, GMM y Spectral
# Para DBSCAN, usamos la solución robusta de eps=1.15

# 1. k-means
set.seed(123) # Para reproducibilidad
res_kmeans_k3 = kmeans(datos_est, centers = 6, nstart = 25)
k_means_clusters = res_kmeans_k3$cluster

# 2. GMM
res_gmm_k3 = Mclust(datos_est, G = 6)
gmm_clusters = res_gmm_k3$classification

# 3. DBSCAN
res_dbscan_eps115 = dbscan(datos_est, eps = 1.2, minPts = 3)
dbscan_clusters = res_dbscan_eps115$cluster

In [ ]:
# Se necesita la matriz de asignación y la matriz de distancias
sil_kmeans = silhouette(k_means_clusters, datos_dist)
plot(sil_kmeans, main = "Silueta (k-means, k=6)", border = NA, col = 2:(3+1)) # colorea hasta k+1 clústeres

In [ ]:
# Alternativa para que tengáis otro gráfico similar (requiere factoextra)
fviz_silhouette(sil_kmeans)

**Interpretación.-** La silueta promedio del modelo ($0.27$) revela que la estructura global de los clústeres es **débil**, indicando que las separaciones entre grupos son modestas y que existe una proximidad considerable entre observaciones de clústeres distintos. Aun así, el análisis permite identificar patrones diferenciados en la calidad interna de cada clúster. En concreto, los clústeres **1**, **4** y **5** muestran siluetas medias entre $0.31$ y $0.35$, lo que sugiere una cohesión razonable y una separación algo más clara respecto al resto. El clúster **5**, con una silueta de $0.35$, es el mejor definido dentro del modelo.

Por el contrario, los clústeres **3** y **6** presentan valores muy bajos de silueta ($0.10$ y $0.12$), indicando una estructura claramente débil: muchas de sus observaciones están más cerca de otros clústeres que del suyo propio. El clúster **2** ocupa una posición intermedia, con una silueta de $0.28$ que refleja una separación moderada pero no sólida. En conjunto, aunque el modelo logra identificar algunos grupos con cierta coherencia interna, la baja silueta promedio sugiere que la partición en seis clústeres no captura una estructura de agrupamiento fuerte en los datos.


In [ ]:
# Generamos y graficamos siluetas para GMM (k=3)
sil_gmm = silhouette(gmm_clusters, datos_dist)
plot(sil_gmm, main = "Silueta (GMM, k=6)", border = NA, col = 2:(3+1))

In [ ]:
# Alternativa para que tengáis otro gráfico similar (requiere factoextra)
fviz_silhouette(sil_gmm)

**Interpretación.-** La silueta promedio del modelo ($0.27$) revela que la estructura global de los clústeres es **débil**, indicando que las separaciones entre grupos son modestas y que existe una proximidad considerable entre observaciones de clústeres distintos. Aun así, el análisis permite identificar patrones diferenciados en la calidad interna de cada clúster. En concreto, los clústeres **1**, **2** y **5** muestran siluetas medias de $0.31$ y $0.32$, lo que sugiere una cohesión razonable y una separación algo más clara respecto al resto. Los clúster **2 y 1**, con una silueta de $0.32$, es los mejores definidos dentro del modelo.

Por el contrario, los clústeres **3** y **6** presentan valores muy bajos de silueta ($0.24$ y $0.12$), indicando una estructura claramente débil: muchas de sus observaciones están más cerca de otros clústeres que del suyo propio. El clúster **4** es un cluster unitario. En conjunto, aunque el modelo logra identificar algunos grupos con cierta coherencia interna, la baja silueta promedio sugiere que la partición en seis clústeres no captura una estructura de agrupamiento fuerte en los datos.


In [ ]:
# Generamos y graficamos siluetas para DBSCAN (eps=1.15)
# Importante: La función silhouette trata al clúster 0 de DBSCAN (ruido) como un clúster separado.
sil_dbscan = silhouette(dbscan_clusters, datos_dist)
plot(sil_dbscan, main = "Silueta (DBSCAN, eps=1.20)", border = NA, col = 2:(max(dbscan_clusters)+1))

In [ ]:
# Alternativa para que tengáis otro gráfico similar (requiere factoextra)
fviz_silhouette(sil_dbscan)

**Interpretación.-** La silueta promedio del modelo ($0.07$) revela que la estructura global de los clústeres ***no se ha encontrado**, indicando que las separaciones entre grupos son grandes y que no existe una proximidad considerable entre observaciones de clústeres distintos. Aun así, el análisis permite identificar patrones diferenciados en la calidad interna de cada clúster. En concreto, los clústeres **1** y **2** muestran siluetas medias entre $0.36$ y $0.53$, lo que sugiere una cohesión razonable y una separación algo más clara respecto al resto. El clúster **2**, con una silueta de $0.53$, es el mejor definido dentro del modelo y siendo el único que posee una estructura aceptable entre todos los modelos.

Por el contrario, el cluster $0$ posee un valor negativo, esto tiene sentido ya que es el formado por los valores outliers.  En conjunto, aunque el modelo logra identificar algunos grupos con cierta coherencia, la baja silueta promedio sugiere que la partición en tres clústeres no captura una estructura de agrupamiento fuerte en los datos.


**Conclusión.-** No se ha podido indentificar un modelo óptimo debido a que entre los criterios no hay ni unidad ni mayoría salbo en el dbsacn el cual nos indica que hay que dividirlo en 10 clusters pero pese a esa unidad los valores de los estadísticos son malos.